# Wave function postprocesing

Here we wish to do postprocesing on Quantum Espresso data extracted using
`pw_export.x` after a band calculation using `ascii=.true.` option. See
https://www.quantum-espresso.org/Doc/INPUT_pw_export.html for more info.

## Checking Orthonormality for Wavefunctions(Fourrier coefficients)

Assume we have our data files, first thing we wish to do is check if wavefunctions are orthonormal. For this we need a procedure that will read from our output the following things: number of points on k-grid and number of wavefunctions (these are actually Fourrier coefficients $C_{nk}^G$ where $n$ is a band index, $k$ is a point in k-space and $G$ is the lattice vector in reciprodal (Fourrier) space).

First, let's make functions that can gather our data for $C_{nk}^G$, ${\bf G}$-vectors, ${\bf k}$-path and one-electron functions eigenvalues $E_{n{\bf{k}}}$: 

In [ ]:
import numpy as np
import re as re

def Gather_C(path,k_point,details=False):    
    FILE = open (path+'/wfc.{}'.format(k_point),"r")
    nbnd=0
    igwx=0
    WF=[]
    if details:
        print("Now performing for k-point={}:\n".format(k_point))
    for line in FILE:
        if 'Info' in line:
            found_nbnd = re.search('nbnd="(.+?)"',line)
            if found_nbnd:
                nbnd = int(found_nbnd.group(1))
        if 'igwx' in line:
            found_igwx=re.search('igwx="(.+?)"',line)
            if found_igwx:
                igwx = int(found_igwx.group(1))
        for n in range(nbnd):
            wf_n=[]
            if '<Wfc.{} type='.format(n+1) in line:
                for i in range(igwx):
                    nextLine=next(FILE)
                    A = (nextLine.replace("\n","")).split(",")
                    C = complex(float(A[0]),float(A[1]))

                    wf_n.append(C)
                WF.append(wf_n)
    if details:
        print("\tReading wave-fuctions: DONE")
    WF=np.array(WF)
    return WF

def Gather_G(path,k_point,details=False):
    FILE = open (path+'/grid.{}'.format(k_point),"r")
    size=0
    GV=[]
    if details:
        print("Gathering G-vectors for k-point={}:\n".format(k_point))
    for line in FILE:
        if '<grid type=' in line:
            size = re.search('size="(.+?)"',line)
            if size:
                size = int(int(size.group(1))/3)
            for i in range(size):
                nextLine=next(FILE)
                g = (nextLine.replace("\n","")).split()
                G_i = np.array([float(g[0]),float(g[1]),float(g[2])])
                GV.append(G_i)
    if details:
        print("\tReading G-vectors: DONE")
    GV=np.array(GV)
    return GV

def Gather_K(path,details=False):
    FILE = open (path+'/index.xml',"r")
    KV=[]
    if details:
        print("Gathering K-vectors...\n")
    for line in FILE:
        if '<Kmesh' in line:
            found_nk = re.search('nk="(.+?)"',line)
            if found_nk:
                nk = int(found_nk.group(1))
        if '<k type=' in line:
            for k in range(nk):
                nextLine=next(FILE)
                k = nextLine.split()
                k_i = [float(k[0]),float(k[1]),float(k[2])]
                KV.append(k_i)
    if details:
        print("\tGathering K-vectors: DONE")
    KV=np.array(KV)
    return KV

def Gather_E(path,details=False,units='Rydberg'):
    FILE = open (path+'/index.xml',"r")
    nbnd=0
    nk=0
    E=[]
    if details:
        print("Now performing for k-point={}:\n".format(k_point))
    for line in FILE:
        if '<Eigenvalues' in line:
            found_nk = re.search('nk="(.+?)"',line)
            if found_nk:
                nk = int(found_nk.group(1))
            found_nbnd = re.search('nbnd="(.+?)"',line)
            if found_nbnd:
                nbnd = int(found_nbnd.group(1))
        for k in range(1,nk+1):
            E_k=[]
            if '<e.{} type='.format(k) in line:
                for n in range(nbnd):
                    nextLine=next(FILE)
                    #print(nextLine.split())
                    #print(nextLine)
                    e = float(nextLine)
                    E_k.append(e)
                E.append(E_k)
    if details:
        print("\tReading wave-fuctions: DONE")
    E=np.array(E)
    if units=='Hartree':
        return E*0.5
    if units=='Electronvolt':
        return E*13.605698066
    if units=='Rydberg':
        return E

Now that we know how to gather data, we can define a function `Check_Orthonormality` that can check if our $C_{nk}^G$ coefficients are indeed orthonormial:

In [ ]:
def Check_Orthonormality(WF, tol=1e-13,details=False):
    nbnd = len(WF)
    Ort_MAT=np.zeros((nbnd,nbnd),dtype=complex)
    for i in range(nbnd):
        for j in range(i,nbnd):
            A=np.array(WF[i])
            B=np.array(WF[j])
            Ort_MAT[i][j]=np.vdot(A,B)
            Ort_MAT[j][i]=Ort_MAT[i][j]
    if details:
        print("\tVector product is: DONE")
    
    orthonormal=True
    tol_global=tol
    index = [0,0]
    for i in range(nbnd):
        if abs(abs(Ort_MAT[i][i])-1.0)<tol:
            pass
        else:
            if details:
                print ("\n\tMatrix element [{},{}] is:{}".format(i,i,Ort_MAT[i][i]))
            orthonormal=False
            ort_new=False
            tol_new=tol
            while ort_new==False:
                    tol_new=tol_new*10
                    if tol_global<tol_new: tol_global=tol_new; index=[i,i]
                    if abs(abs(Ort_MAT[i][i])-1.0)<tol_new:
                        if details:
                            print("\tTolerance for {}".format(tol_new))
                        ort_new=True
    for i in range(nbnd):
        for j in range(i+1,nbnd):
            if abs(Ort_MAT[i][j])<tol:
                pass
            else:
                if details:
                    print ("\n\tMatrix element: {},{} is:{}".format(i,j,abs(Ort_MAT[i][j])))
                orthonormal=False
                ort_new=False
                tol_new=tol
                while ort_new==False:
                    tol_new=tol_new*10
                    if tol_global<tol_new: tol_global=tol_new; index=[i,j]
                    if abs(Ort_MAT[i][j])<tol_new:
                        if details: 
                            print("\tTolerance for {}".format(tol_new))
                        ort_new=True                    
    print('\nOrthonormality {} for tolerance {}'.format(orthonormal,tol))
    if not orthonormal:
        print('Orthonormal for tolerance {} in index {}\n'.format(tol_global,index))

We can test these functions for $n_{bands} = 20, 40, 60, 80, 100, 120, 140$:

In [ ]:
for nbnd in ['20', '40' , '60' , '80', '100' , '120' , '140']:
    path = "/home/mjocic/data_paradox/nbnd_"+nbnd
    tolerance=1e-13
    print('\n\t****FOR N_bands = '+nbnd+' ******')
    for k in range(1,42):
        print('\nk-point number {}'.format(k))
        C = Gather_C(path,k)
        Check_Orthonormality(C,tolerance)

## Calculating the $k \cdot p$ Hamiltonian

### Obtaining the $k\cdot p$ Matrix Elements

Now that we have collected our $C_{nk}^G$ coefficients we can proceed to find matrix elements of $\langle m |{\bf k}\cdot {\bf p} | n \rangle$ where
$m$ and $n$ represent the band index and ${\bf k}$ and ${\bf p}$ represent a certain k-point in Fourrier space and momentum operator respectively. Starting from
$$
    \int d^3 {\bf r} u_{mk}^* {\bf k} \cdot {\bf p} u_{nk}
$$
we arrive to the expression:
$$
    i {\bf k} \cdot \sum_G (C_{mk}^G)^* {\bf G} C_{nk}^G
$$
which will need to calculate. We can see that we have one part of the expression but we need to incorporate $G$-vectors as well. This is done by reading the `grid.*` files and extracting data considering G-points. The procedure will be similar to the one for orthonormality check, except we need to insert ${\bf G}$-vectors that we obtained from `grid.*` file.

Eigenproblem that we're trying to solve is:
$$
H_{\bf k}({\bf r})u_{n{\bf k}}({\bf r})=E_{n {\bf k}} u_{n{\bf k}}({\bf r})
$$
Where we can expand functions $u_{n{\bf k}}$ around a certain k-point, since Bloch functions form an orhonormial basis, which corresponds to some eigenvector with eigenvalue $E_{n {\bf k}}$. For our case, let that be
the R symmetry point ${\bf k}_R$ so that we can write:
$$
u_{n{\bf k}}({\bf r}) = \sum_n C_{nk} u_{nR}({\bf r})
$$
and use it to obtain:
$$
\sum_n \left[ E_{nR} + 
    \frac{\hbar^2}{2 m_0}(k^2-k_R^2) \right] C_{nR} \delta_{mn}
    +\sum_n \frac{\hbar}{m_0}
\left\langle u_{mR}\left|({\bf k}-{\bf k}_R)\cdot {\bf p } \right|u_{nR}\right\rangle C_{nk} = \sum_n E_{n\bf{k}} C_{nk} \delta_{mn}
.
$$
This can be represented in matrix form where $h_{mn}$ are matrix coefficients of Hamiltonian which we now need do diagonalize:
$$
h_{mn}=\left[E_{nR} + \frac{\hbar^2}{2 m_0}(k^2-k_R^2) \right] \delta_{mn}
+\sum_n \frac{\hbar}{m_0}
\left\langle u_{mR}\left|({\bf k}-{\bf k}_R)\cdot {\bf p } 
\right| u_{nR}\right\rangle .
$$
We focus now on obtaining the Hamiltonian matrix $h_{mn}$, then diagonalizing it and obtaining the eigenvalues.

In [ ]:
import numpy as np
import re as re

def Compute_Matrix_Element(G_vec,C_terms,m,n):
    A = np.array(C_terms[m])
    B = np.array(C_terms[n])
    Gx=G_vec[:,0]
    Gy=G_vec[:,1]
    Gz=G_vec[:,2]
    G = [np.vdot(A,Gx*B),np.vdot(A,Gy*B),np.vdot(A,Gz*B)]
    return np.array(G)  

def Compute_H(k_mesh,k_point,E_k,
              G_space,C_terms,a,
              units='Rydberg',
              details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    kR2= np.dot(k_point,k_point)
    H=[]
    for k_i in range(len(k_mesh)):
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((len(C_terms),len(C_terms)),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                CGC = Compute_Matrix_Element(G_space,C_terms,m,n)
                KdP = np.dot(kkR,CGC)
                if m==n:
                    H_i[m][m] = E_k[m] + 0.5*tpiba2*(k2-kR2) + tpiba2*KdP
                else:
                    H_i[m][n] = tpiba2*KdP
        H_i = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        if details:
            print("eigenvalues of H_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i)*uni[1])
    print('Calculating bands: DONE')
    return np.array(H)*uni[1]

In [ ]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
E=Gather_E(path,units='Hartree')
E_R = E[k_R-1]
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]
G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)

In [ ]:
H = Compute_H(k_points,k_Rpoint,E_R,
              G_R,C_R,a=10.8,
              units='Hartree',
              details=True)

 Here, we insert a small code for printing our results to files
 `plotband_kp.dat` and `plotband_dft.dat` for ${\bf k}\cdot {\bf p}$ and DFT calculation, respectively:

In [ ]:
k2=[]
FILE = open ('/home/mjocic/Desktop/data_export/nbnd_120'+'/bands_gnuplt',"r")
for line in FILE:
    if line is not '\n':
        kgnu=line.split()[0]
        k2.append(float(kgnu))
    else:
        break;
FILE.close()
k2=np.array(k2)
x=[];y_kp=[];y_dft=[]
FILE_kp = open( path+'/plotband_kp.dat','w')
FILE_dft = open( path+'/plotband_dft.dat', 'w')
for n in range(H.shape[1]):
    for k_i in range(len(k2)):
        x.append(k2[k_i])
        y_kp.append(H[k_i][n].real);
        y_dft.append(E[k_i][n])
        FILE_kp.write("{} {}\n".format (k2[k_i],H[k_i][n].real))
        FILE_dft.write("{} {}\n".format (k2[k_i],E[k_i][n].real))
    FILE_kp.write("\n")
    FILE_dft.write("\n")
FILE_kp.close()
FILE_dft.close()

import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_kp,label='$k \cdot p$')
plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
plt.ylim(-0.1,0.3)
plt.xticks([0,0.707,1.207,1.9142,2.4142],('$\Gamma$','M','X','R','M'))
plt.ylabel('E(k) [Hartree]')
plt.legend()
plt.show()

Having seen that we indeed have orthonormality for our one-electron functions from DFT calculation, we can move forward to checking energy levels in certain ${\bf k}$-points and their degeneracies.

We need to see what kind of symmetries are observed at a given ${\bf k}$-point. To do this we first need to obtain from our numerical calculations how many times is a certain band degenerated at that point and then find on which reducible representations does the certain point decompose.

In [ ]:
print(type(E[30]),E[30].shape)
print(E[30,2])
deg_e=[]
deg_h=[]
index=0
while index<E[30].shape[0]:
    if abs(E[30,index]-E[30,index+1])<1e-4:
        index += 1
        if abs(E[30,index]-E[30,index+1])<1e-4:
            index += 1
            deg_e.append([E[30,index],3])
        else:
            deg_e.append([E[30,index],2])
    else:
        deg_e.append([E[30,index],1])
    index +=1
deg_e = np.array(deg_e)
index=0
while index<H[30].shape[0]:
    if abs(H[30,index]-H[30,index+1])<1e-4:
        index += 1
        if abs(H[30,index]-H[30,index+1])<1e-4:
            index += 1
            deg_h.append([H[30,index],3])
        else:
            deg_h.append([H[30,index],2])
    else:
        deg_h.append([H[30,index],1])
    index +=1
deg_h = np.array(deg_h)
for i in range(len(deg_e)):
    print(deg_e[i],deg_h[i])

In [ ]:
print(len(C_R[1]))
#print(len(G_R))
print(np.vdot(C_R[3],C_R[5]))
print(np.vdot(C_R[1],C_R[1]) + np.vdot(C_R[2],C_R[2]))
print(np.vdot(C_R[3],C_R[3]) + np.vdot(C_R[4],C_R[4]) + np.vdot(C_R[5],C_R[5]))

In [ ]:
from post_procesing_tools import *

A = np.array([[1,0,0],
              [0,-1,0],
              [0,0,-1]])
X = np.array([2,3,1])
np.matmul(A,X)

path = "/home/mjocic/data_paradox/nbnd_120"
C_R = Gather_C(path,31)
G_R = Gather_G(path,31)

In [ ]:
import re as re

class Symmetry:
    def __init__(self, matrix, operation):
        self.matrix = matrix
        self.operation = operation
        self.char = np.trace(matrix)
    def __str__(self):
        return '{} {}\n'.format(self.matrix,self.operation)
    def __repr__(self):
        return '{} {}\n'.format(self.matrix,self.operation)

class Charachters:
    def __init__(self, charachter, operation):
        self.charachter =self[:,0]
        self.operations =self[:,1]
    def __str__(self):
        return '{} {}\n'.format(self.charachter,self.operation)
    def __repr__(self):
        return '{} {}\n'.format(self.charachter,self.operation) 
        
def Gather_Symmetries(path):
    FILE = open (path+'/index.xml',"r")
    n_sym=0
    inv_sym = False
    sym_list=[]
    for line in FILE:
        if '<symmops' in line:
            n_sym = re.search('nsym="(.+?)"',line)
            if n_sym:
                n_sym = int(n_sym.group(1))
    for i in range(1,n_sym+1):
        FILE = open (path+'/index.xml',"r")
        for line in FILE:
            if '<info.{} name'.format(i) in line:
                sym_name = re.search('name="(.+?)"',line)
                sym_name = str(sym_name.group(1))
                nextLine=next(FILE)
                mat_type = re.search('type="(.+?)"',nextLine)
                nextLine=next(FILE)
                r = nextLine.split()
                R1 = [int(r[0]),int(r[1]),int(r[2])]
                nextLine=next(FILE)
                r = nextLine.split()
                R2 = [int(r[0]),int(r[1]),int(r[2])]
                nextLine=next(FILE)
                r = nextLine.split()
                R3 = [int(r[0]),int(r[1]),int(r[2])]
                mat = np.array([R1,R2,R3])
                R = Symmetry(mat,sym_name)
                sym_list.append(R)
    return sym_list

In [ ]:
symmetry_list = Gather_Symmetries(path)

#def Reduce_to_Class(symmetry_list):
for i in range(len(symmetry_list)):
    print(i+1,'\t', symmetry_list[i].char, symmetry_list[i].operation,
         '\n', symmetry_list[i].matrix)

In [ ]:
print(G_R[207])
print(np.matmul(symmetry_list[4].matrix,G_R[207]))
print(G_R[207,2])
print(C_R[119,121])
len(G_R)==len(C_R[3])

In [ ]:
import numpy as np
k_start=2
k_stop=4
charachters=[]
for s in [0, 1, 4, 6, 16, 24, 25, 28, 30, 40]:
    sym_name=symmetry_list[s].operation
    sym_matrix=symmetry_list[s].matrix
    print('Now performing for '+ sym_name)
    if s==0:
        char = 3
        charachters.append([char,sym_name])
        print('charachter = ',char)
    elif s==24:
        char = 3
        charachters.append([char,sym_name])
        print('charachter = ',char)
    else:
        char = 0
        for k in range(k_start,k_stop+1):
            RC_R = np.copy(C_R[k])
            for i in range(len(G_R)):
                g_i = np.matmul(sym_matrix,G_R[i])
                not_found = Talse
                index = 0
                while not_found:
                    if g_i[0] == G_R[index,0] and g_i[1]==G_R[index,1] and g_i[2]==G_R[index,2]:
                        RC_R[i],RC_R[j] = RC_R[j],RC_R[i]
            char += np.vdot(RC_R,C_R[k])
        print('charachter = ',char)
    charachters.append([char, sym_name])

In [ ]:
import numpy as np
k_start=2
k_stop=4
charachters=[]
for s in range(len(symmetry_list)):
    sym_name=symmetry_list[s].operation
    sym_matrix=symmetry_list[s].matrix
    print('\nNow performing for '+ sym_name)
    char=0
    for k in range(8,9):
        RC_R = np.zeros(len(G_R),dtype=complex)
        for i in range(len(G_R)):
            g_i = np.matmul(sym_matrix,G_R[i]-k_R)
            for j in range(len(G_R)):
                if g_i[0]==G_R[j,0] and g_i[1]==G_R[j,1] and g_i[2]==G_R[j,2]:
                    RC_R[i] = C_R[k,j]
        char += np.vdot(RC_R,C_R[k])
    print('charachter = ',char)
    charachters.append([char, sym_name])

In [6]:
import numpy as np

char_G  = np.array([3,-1,1,-1,0,-3,1,-1,1,0])

char_g1 = np.array([1,1,1,1,1,1,1,1,1,1])
char_g2 = np.array([1,1,-1,-1,1,1,1,-1,-1,1])
char_g12= np.array([2,2,0,0,-1,2,2,0,0,-1])
char_g15= np.array([3,-1,1,-1,0,-3,1,-1,1,0])
char_g25= np.array([3,-1,-1,1,0,-3,1,1,-1,0])

char_g1p= np.array([1, 1, 1, 1, 1,-1,-1,-1,-1,-1])
char_g2p= np.array([1, 1,-1,-1, 1,-1,-1, 1, 1,-1])
char_g12p=np.array([2, 2, 0, 0,-1,-2,-2, 0, 0, 1])
char_g15p=np.array([3,-1, 1,-1, 0, 3,-1, 1,-1, 0])
char_g25p=np.array([3,-1,-1, 1, 0, 3,-1,-1, 1, 0])

Nk = [1,3,6,6,8,1,3,6,6,8]

In [56]:
def char_coef(g_ired,g_red,Nk):
    prod=0
    for i in range(len(Nk)):
        prod+=g_ired[i]*g_red[i]*Nk[i]
    return prod
        

In [8]:
for char_g in [char_g1,char_g2,char_g12,char_g15,char_g25]:
    print(char_coef(char_g,char_G,Nk)/48)

0.0
0.0
0.0
1.0
0.0


In [9]:
for char_g in [char_g1p,char_g2p,char_g12p,char_g15p,char_g25p]:
    print(char_coef(char_g,char_G,Nk)/48)

0.0
0.0
0.0
0.0
0.0


In [ ]:
#for i in range(len(RC_R)):
#    print(RC_R[i].real,'\t', C_R[8,i].real)
print(np.sum(RC_R),'\n',np.sum(C_R[8]))
print(len(RC_R), len(C_R[8]), len(G_R))

In [ ]:
k_start=2
k_stop=4
charachters=[]
for s in [0, 1, 4, 6, 16, 24, 25, 28, 30, 40]:
    sym_name=symmetry_list[s].operation
    sym_matrix=symmetry_list[s].matrix
    print('\nNow performing for '+ sym_name)
    char=0
    for k in range(k_start,k_stop+1):
        RC_R = np.zeros(len(G_R),dtype=complex)
        for i in range(len(G_R)):
            g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
            not_found = True
            j=0
            while not_found:
                if j>=len(G_R):
                    print('Error')
                if g_i[0]==G_R[j,0] and g_i[1]==G_R[j,1] and g_i[2]==G_R[j,2]:
                    RC_R[i] = C_R[k,j]
                    not_found = False
                else: j+=1
                if j==len(G_R) and not_found:
                    print('not found error')
                    RC_R[i] = 0.+0j
                    not_found = False
                    print('g[{}]={}'.format(i,g_i))
        char += np.vdot(RC_R,C_R[k])
    print('charachter = ',char)
    charachters.append([char, sym_name])

In [36]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
#E=Gather_E(path,units='Hartree')
#E_G = E[k_G-1]
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]
G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)
symmetry_list = Gather_Symmetries(path)

In [49]:
# New and improved charachter calculation:
k_start=2
k_stop=4
charachters=[]
dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
print(dim_G)
M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
print('Creating G-matrix')
for i in range(len(G_R)):
    n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
#    print(n1,n2,n3, i)
    M[n1,n2,n3] = i
print('G-matrix: Done')

for s in [0, 1, 6, 4, 16, 24, 25, 30, 28, 40]:
    sym_name=symmetry_list[s].operation
    sym_matrix=symmetry_list[s].matrix
    print('\nNow performing for '+ sym_name)
    char=0
    for k in range(k_start,k_stop+1):
        RC_R = np.zeros(len(G_R),dtype=complex)
        for i in range(len(G_R)):
            g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
            n1,n2,n3=int(g_i[0]),int(g_i[1]),int(g_i[2])
            index = M[n1,n2,n3]
            RC_R[i] = C_R[k,index]
        char += np.vdot(RC_R,C_R[k])
    print('charachter = ',char)
    charachters.append([int(round(char.real)), sym_name])

print('Done')

31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (2.9999999999999982+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (-1.0000000000000062-1.7044778484893772e-16j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (0.9999999999999973+1.609823385706477e-15j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-0.9999999999999986+4.483009554488315e-17j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (3.3306690738754696e-16+1.1102230246251565e-15j)

Now performing for inversion
charachter =  (-3-2.7350089611403888e-17j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (1.0000000000000053+1.149290235560444e-16j)

Now performing for inv.  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-0.9999999999999974-1.1657341758564144e-15j)

Now performing for inv. 180 deg rotation - cart. axis [1,1,0]
charachter =  (0.9999999999999989+8.3347

In [65]:
print(np.array(charachters))
char_G = np.array(charachters)
char_G = np.array(char_G[:,0],dtype=int)
print(type(char_G[0]))
char_g1 = np.array([1,1,1,1,1,1,1,1,1,1])
char_g2 = np.array([1,1,-1,-1,1,1,1,-1,-1,1])
char_g12= np.array([2,2,0,0,-1,2,2,0,0,-1])
char_g15= np.array([3,-1,1,-1,0,-3,1,-1,1,0])
char_g25= np.array([3,-1,-1,1,0,-3,1,1,-1,0])

char_g1p= np.array([1, 1, 1, 1, 1,-1,-1,-1,-1,-1])
char_g2p= np.array([1, 1,-1,-1, 1,-1,-1, 1, 1,-1])
char_g12p=np.array([2, 2, 0, 0,-1,-2,-2, 0, 0, 1])
char_g15p=np.array([3,-1, 1,-1, 0, 3,-1, 1,-1, 0])
char_g25p=np.array([3,-1,-1, 1, 0, 3,-1,-1, 1, 0])

Nk = [1,3,6,6,8,1,3,6,6,8]

for char_g in [char_g1,char_g2,char_g12,char_g15,char_g25]:
    print(char_coef(char_g,char_G,Nk)/48)

[['3' 'identity']
 ['-1' '180 deg rotation - cart. axis [0,0,1]']
 ['1' ' 90 deg rotation - cart. axis [0,0,-1]']
 ['-1' '180 deg rotation - cart. axis [1,1,0]']
 ['0' '120 deg rotation - cart. axis [-1,-1,-1]']
 ['-3' 'inversion']
 ['1' 'inv. 180 deg rotation - cart. axis [0,0,1]']
 ['-1' 'inv.  90 deg rotation - cart. axis [0,0,-1]']
 ['1' 'inv. 180 deg rotation - cart. axis [1,1,0]']
 ['0' 'inv. 120 deg rotation - cart. axis [-1,-1,-1]']]
<class 'numpy.int64'>
0.0
0.0
0.0
1.0
0.0


In [ ]:
norm_G = []
norm_C = []
for i in range(len(G_R)):
    norm_G.append(np.dot(G_R[i],G_R[i]))
    norm_C.append(np.vdot(C_R[8,i],C_R[8,i]))
norm_G = np.sqrt(np.array(norm_G))
norm_C = np.sqrt(np.array(norm_C))
print(norm_G)
print(norm_C)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(norm_G.real,norm_C.real)
plt.xlim(0,6)
plt.show()

In [ ]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_G=1
E=Gather_E(path,units='Hartree')
E_G = E[k_G-1]
k_points= Gather_K(path)
k_Gpoint = k_points[k_G-1]
G_G = Gather_G(path,k_G)
C_G = Gather_C(path,k_G)

In [ ]:
H = Compute_H(k_points,k_Gpoint,E_G,
              G_G,C_G,a=10.8,
              units='Hartree',
              details=True)

In [ ]:
print(type(E[0]),E[0].shape)
print(E[0,2])
deg_e=[]
deg_h=[]
index=0
while index+3<E[0].shape[0]:
    if abs(E[0,index]-E[0,index+1])<1e-4:
        index += 1
        if abs(E[0,index]-E[0,index+1])<1e-4:
            index += 1
            deg_e.append([E[0,index],3])
        else:
            deg_e.append([E[0,index],2])
    else:
        deg_e.append([E[0,index],1])
    index +=1
deg_e = np.array(deg_e)
index=0
while index+3<H[0].shape[0]:
    if abs(H[0,index]-H[0,index+1])<1e-4:
        index += 1
        if abs(H[0,index]-H[0,index+1])<1e-4:
            index += 1
            deg_h.append([H[0,index],3])
        else:
            deg_h.append([H[0,index],2])
    else:
        deg_h.append([H[0,index],1])
    index +=1
deg_h = np.array(deg_h)
for i in range(len(deg_e)):
    print(deg_e[i],deg_h[i])

In [ ]:
symmetry_list = Gather_Symmetries(path)

#def Reduce_to_Class(symmetry_list):
for i in range(len(symmetry_list)):
    print(i+1,'\t', symmetry_list[i].char, symmetry_list[i].operation,
         '\n', symmetry_list[i].matrix)

In [ ]:
k_start=2
k_stop=4
charachters=[]
for s in range(len(symmetry_list)):
    sym_name=symmetry_list[s].operation
    sym_matrix=symmetry_list[s].matrix
    print('\nNow performing for '+ sym_name)
    char=0
    for k in range(5,6):
        RC_G = np.zeros(len(G_G),dtype=complex)
        for i in range(len(G_G)):
            g_i = np.matmul(sym_matrix,G_G[i])
            not_found = True
            j=0
            while not_found:
                if j>=len(G_G):
                    print('Error')
                if g_i[0]==G_G[j,0] and g_i[1]==G_G[j,1] and g_i[2]==G_G[j,2]:
                    RC_G[i] = C_G[k,j]
                    not_found = False
                else: j+=1
                if j==len(G_G) and not_found:
                    print('not found error')
                    RC_G[i] = 0.+0j
                    not_found = False
                    print('g[{}]={}'.format(i,g_i))
        char += np.vdot(RC_G,C_G[k])
    print('charachter = ',char)
    charachters.append([char, sym_name])

In [ ]:
uk_start=2
uk_stop=4
charachters=[]
for s in range(len(symmetry_list)):
    sym_name=symmetry_list[s].operation
    sym_matrix=symmetry_list[s].matrix
    print('\nNow performing for '+ sym_name)
    char=0
    for k in range(uk_start,uk_stop+1):
        RC_G = np.zeros(len(G_G),dtype=complex)
        for i in range(len(G_G)):
            g_i = np.matmul(sym_matrix,G_G[i])
            not_found = True
            j=0
            while not_found:
                if j>=len(G_G):
                    print('Error')
                if g_i[0]==G_G[j,0] and g_i[1]==G_G[j,1] and g_i[2]==G_G[j,2]:
                    RC_G[i] = C_G[k,j]
                    not_found = False
                else: j+=1
                if j==len(G_G) and not_found:
                    print('not found error')
                    RC_G[i] = 0.+0j
                    not_found = False
                    print('g[{}]={}'.format(i,g_i))
        char += np.vdot(RC_G,C_G[k])
    print('charachter = ',char)
    charachters.append([char, sym_name])

In [ ]:
char_G  = np.array([3,-1,-1,1,0,3,-1,-1,1,0])

char_g1 = np.array([1,1,1,1,1,1,1,1,1,1])
char_g2 = np.array([1,1,-1,-1,1,1,1,-1,-1,1])
char_g12= np.array([2,2,0,0,-1,2,2,0,0,-1])
char_g15= np.array([3,-1,1,-1,0,-3,1,-1,1,0])
char_g25= np.array([3,-1,-1,1,0,-3,1,1,-1,0])

char_g1p= np.array([1, 1, 1, 1, 1,-1,-1,-1,-1,-1])
char_g2p= np.array([1, 1,-1,-1, 1,-1,-1, 1, 1,-1])
char_g12p=np.array([2, 2, 0, 0,-1,-2,-2, 0, 0, 1])
char_g15p=np.array([3,-1, 1,-1, 0, 3,-1, 1,-1, 0])
char_g25p=np.array([3,-1,-1, 1, 0, 3,-1,-1, 1, 0])

Nk = [1,3,6,6,8,1,3,6,6,8]

In [ ]:
for char_g in [char_g1,char_g2,char_g12,char_g15,char_g25]:
    print(char_coef(char_g,char_G,Nk)/48)

In [ ]:
for char_g in [char_g1p,char_g2p,char_g12p,char_g15p,char_g25p]:
    print(char_coef(char_g,char_G,Nk)/48)

In [83]:
def Compute_Matrix_Element2(G_vec,C_terms,E_terms,kkr,m):
        cgc = 0
        for i in range(len(E_terms)):
            if abs(E_terms[i]-E_terms[m])<1e-11: continue
            delta_E1 =1/(E_terms[m] - E_terms[i])
            if abs(delta_E1)>1e-2: continue
            kp = np.dot(kkr,Compute_Matrix_Element(G_vec,C_terms,m,i))
            cgc += delta_E1*np.vdot(kp,kp)
        return cgc
            

def Compute_H3(k_mesh,k_point,E_k,
              G_space,C_terms,a,
              units='Hartree',
              details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    kR2= np.dot(k_point,k_point)
    H=[]
    H_eig = []
    for k_i in range(len(k_mesh)):
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((len(C_terms),len(C_terms)),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                CGC = Compute_Matrix_Element(G_space,C_terms,m,n)
                KdP = np.dot(kkR,CGC)
                if m==n:
                    KdP2 = Compute_Matrix_Element2(G_space,C_terms,E_k,kkR,m)
                    H_i[m][m] = E_k[m] + 0.5*tpiba2*(k2-kR2) + tpiba2*KdP + tpiba2**2*KdP2
                else:
                    H_i[m][n] = tpiba2*KdP
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of H_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i)*uni[1])
    print('Calculating bands: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

In [84]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_20"
k_R=31
E=Gather_E(path,units='Hartree')
E_R = E[k_R-1]
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]
G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)

In [85]:
Ham2, Ham2eig = Compute_H3(k_points,k_Rpoint,E_R,G_R,C_R,a=10.8)

Calculating bands: DONE


In [64]:
Hameig = Compute_H(k_points,k_Rpoint,E_R,G_R,C_R,a=10.8,units='Hartree')

Calculating bands: DONE


In [86]:
k2=[0.0]
d=0
for i in range(len(k_points)-1):
    d1 = k_points[i+1]-k_points[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)
x=[];y_kp=[];y_dft=[];y_kp2=[];
#FILE_kp = open( path+'/plotband_kp.dat','w')
#FILE_dft = open( path+'/plotband_dft.dat', 'w')
for n in range(Ham2eig.shape[1]):
    for k_i in range(len(k2)):
        x.append(k2[k_i])
        y_kp.append(Hameig[k_i][n].real*0.5);
        y_kp2.append(Ham2eig[k_i][n].real)
        y_dft.append(E[k_i][n])

In [89]:
# Plot the results
import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_kp,label='$k \cdot p$')
plt.plot(x,y_kp2,label='$k \cdot p 2$')
plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
#plt.ylim(y_dft[0],y_dft[1500])
plt.ylim(-0.2,0.3)
plt.xticks([k2[0],k2[10],k2[20],k2[30],k2[40]],('$\Gamma$','M','X','R','M'))
plt.ylabel('E(k) [Hartree]')
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


In [103]:
print(k_points[20:40].shape)

(20, 3)


In [409]:
def Compute_H2(k_mesh,k_point,E_k,
              G_space,C_terms,a,
              units='Hartree',
              details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    kR2= np.dot(k_point,k_point)
    H=[]
    H_eig = []
    for k_i in range(len(k_mesh)):
        print('point=',k_i)
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((len(C_terms),len(C_terms)),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                CGC = Compute_Matrix_Element(G_space,C_terms,m,n)
                KdP = np.dot(kkR,CGC)
                KdP2 = Compute_Perturbation(G_space,C_terms,E_k,kkR,m,n)
                if m==n:
                    H_i[m][m] = E_k[m] + 0.5*tpiba2*(k2-kR2) + tpiba2*KdP + tpiba4*KdP2
                else:
                    H_i[m][n] = tpiba2*KdP + tpiba4*KdP2
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of H_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i)*uni[1])
    print('Calculating bands: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

In [465]:
def Compute_Perturbation(G_vec,C_terms,ext_basis,E_terms,E_ext,kkr,m,n):
        Eavg = (E_terms[m]+E_terms[n])/2
        new_basis = np.vstack((ext_basis,C_terms))
        dim = len(ext_basis)
        cgc = 0
        for l in range(len(E_ext)):
            #if abs(E_ext[l]-Eavg)<1e-10: 
                #cont_count +=1
                #print('continues<',cont_count);
                #continue
            delta_E1 =1/(Eavg - E_ext[l])
            #if abs(delta_E1)>1e-1:
                #print('continues>');
                #continue
            kpi = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis,dim+m,l))
            kpj = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis,l,dim+n))
            cgc += kpi*kpj*delta_E1
        #print(cgc)
        #if np.absolute(cgc)<1e-10: return 0.0
        #else:
        return cgc

def Compute_H2_test(k_mesh,k_point,E_k,E_ext,
              G_space,C_terms,ext_basis,a,
              units='Hartree',
              details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    c=137
    tpiba22 = tpiba4/(c**2)
    kR2= np.dot(k_point,k_point)
    H=[]
    H_eig = []
    H1 = []
    dim = len(C_terms)
    Index = np.arange(dim**2).reshape((dim,dim)).T
    for m in range(dim):
        for n in range(dim):
            H1.append(Compute_Matrix_Element(G_space,C_terms,m,n))
    for k_i in range(len(k_mesh)):
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((len(C_terms),len(C_terms)),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                index = Index[m,n]
                CGC = H1[index]
                KdP = np.dot(kkR,CGC)
                KdP2 = Compute_Perturbation_test(G_space,C_terms,ext_basis,E_k,E_ext,kkR,m,n)
                if m==n:
                    H_i[m][m] = E_k[m] + 0.5*tpiba2*(k2-kR2) + tpiba2*KdP + tpiba4*KdP2
                else:
                    H_i[m][n] = tpiba2*KdP + tpiba4*KdP2
        H_i_eig = (np.linalg.eigvals(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of H_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating bands: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

In [245]:
#from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]

G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)
symmetry_list = Gather_Symmetries(path)

k_start=18
k_stop=20
g25p=Gather_Matrices(k_start,k_stop,symmetry_list,G_R,C_R,k_Rpoint)

path2="/home/mjocic/data_paradox"
GM_25p = Gather_Symmetries(path2,"G25p_t2g.xml")

r = Compute_small_r(matrices,G_25p)
U_g25p = Gather_Unitary_Transforms(r,matrices,G_25p)

u_11 = U_g25p[0,0]
u_11i= nplin.inv(u_11)

order=48
dim=3
order=48
dim=3


In [213]:
print ( u_11.dot(np.transpose(np.conjugate(u_11))) )

[[ 1.00000000e+00-2.18996922e-18j  6.24500451e-16+2.46330734e-16j
   5.27355937e-16+6.93889390e-16j]
 [ 6.24500451e-16-1.87350135e-16j  1.00000000e+00-7.19602000e-18j
  -2.17534324e-15-3.88578059e-16j]
 [ 5.27355937e-16-6.93889390e-16j -2.17534324e-15+4.44089210e-16j
   1.00000000e+00+2.02987730e-18j]]


In [26]:
u_11a = np.transpose(np.conjugate(u_11))
print(u_11.dot(u_11a))
norm = np.trace(u_11a.dot(u_11))/len(u_11)
u_11 = u_11/np.sqrt(norm)
u_11a= u_11a/np.sqrt(norm)

[[ 2.56000000e+02-5.60632119e-16j  1.59872116e-13+6.30606678e-14j
   1.35003120e-13+1.77635684e-13j]
 [ 1.59872116e-13-4.79616347e-14j  2.56000000e+02-1.84218112e-15j
  -5.56887869e-13-9.94759830e-14j]
 [ 1.35003120e-13-1.77635684e-13j -5.56887869e-13+1.13686838e-13j
   2.56000000e+02+5.19648588e-16j]]


In [364]:
basis_qe=C_R[18:21:1]
basis_g25p = u_11.T.dot(basis_qe)

In [422]:
np.vdot(basis_g25p[0],basis_g25p[0])/256
#print('{}\n{}'.format(u_11/16,u_11i*16))
print(np.sqrt(norm))
basis_g25p=basis_g25p/16

(16.00000000000014+0j)


In [423]:
path = "/home/mjocic/data_paradox/nbnd_120"
E = Gather_E(path,units='Hartree')
E_R = E[30]
basis4 = np.array([C_R[17],basis_g25p[0],basis_g25p[1],basis_g25p[2]])
Energy4 = np.array([E_R[17],E_R[18],E_R[19],E_R[20]])
k_mesh4 = k_points [20:41]

In [466]:
#extended basis for H2:
#print(C_R[:17].shape)
#print(C_R[17:21].shape)
#print(C_R[21:].shape)
#print(C_R[:17].shape[0]+C_R[17:21].shape[0]+C_R[21:].shape[0])
basis_ext = np.vstack((C_R[:17],C_R[21:]))
E_ext = np.append(E_R[:17],E_R[21:])
basis4_qe= C_R[17:21]
print(basis_ext.shape)
print(E_ext.shape)
print(basis4.shape)

(116, 15192)
(116,)
(4, 15192)


In [363]:
print(u_11.dot(u_11a)/16)

[[ 1.00000000e+00-1.29520556e-18j  6.10622664e-16+1.66533454e-16j
   5.55111512e-16+6.93889390e-16j]
 [ 6.24500451e-16-1.94289029e-16j  1.00000000e+00-2.44611405e-17j
  -2.18054741e-15-4.44089210e-16j]
 [ 4.99600361e-16-7.07767178e-16j -2.19876201e-15+3.88578059e-16j
   1.00000000e+00+2.22854739e-17j]]


In [197]:
full_basis_qe = np.vstack((basis_ext,basis4_qe))
full_basis_g25= np.vstack((basis_ext,basis4))

In [238]:
l = -4
a3 = Compute_Matrix_Element(G_R,full_basis_qe,l,-1)
a2 = Compute_Matrix_Element(G_R,full_basis_qe,l,-2)
a1 = Compute_Matrix_Element(G_R,full_basis_qe,l,-3)
matrix_elements_1 = np.array([a1,a2,a3])
b3 = Compute_Matrix_Element(G_R,full_basis_g25,l,-1)
b2 = Compute_Matrix_Element(G_R,full_basis_g25,l,-2)
b1 = Compute_Matrix_Element(G_R,full_basis_g25,l,-3)
matrix_elements_2 = np.array([b1,b2,b3])
mat_transformed_12 = u_11.T.dot(matrix_elements_1)
mat_transformed_21 = u_11a.T.dot(matrix_elements_2)

In [239]:
eps = 1e-15
print("mat1=\n{}\nmat2=\n{}\n".format(matrix_elements_1,matrix_elements_2))
print("mat21=\n{}\nmat12=\n{}\n".format(mat_transformed_21,mat_transformed_12))
print("\t matrix 1: \n")
for m in range(len(matrix_elements_1)):
    for n in range(len(matrix_elements_1)):
        a = mat_transformed_21[m,n]
        b = matrix_elements_1[m,n]
        print("[{},{}]:{}\n".format(m,n,abs(a.real-b.real)<eps and abs(a.imag-b.imag)<eps))
print("\t matrix 2: \n")
for m in range(len(matrix_elements_2)):
    for n in range(len(matrix_elements_2)):
        a = mat_transformed_12[m,n]
        b = matrix_elements_2[m,n]
        print("[{},{}]:{}\n".format(m,n,abs(a.real-b.real)<eps and abs(a.imag-b.imag)<eps))

mat1=
[[ 0.26634936-0.20792942j  0.06355563+0.04194552j -0.33515652+0.38637587j]
 [ 0.44452084+0.03914511j -0.20938823-0.2775332j   0.24811558-0.00556916j]
 [-0.03872225+0.25842683j -0.39787592+0.31083674j  0.04584511+0.23723667j]]
mat2=
[[ 1.82769449e-10+2.83320910e-12j  1.93887805e-10+1.61457010e-10j
  -4.04775990e-01+4.66634738e-01j]
 [-4.04775991e-01+4.66634739e-01j  2.53606342e-10+2.33358129e-10j
   1.67378738e-10+1.60664480e-10j]
 [ 7.62305094e-11-2.82586079e-10j -4.04775991e-01+4.66634739e-01j
  -3.55487969e-10-1.55858761e-10j]]

mat21=
[[ 0.26634936-0.20792942j  0.06355563+0.04194552j -0.33515652+0.38637587j]
 [ 0.44452084+0.03914511j -0.20938823-0.2775332j   0.24811558-0.00556916j]
 [-0.03872225+0.25842683j -0.39787592+0.31083674j  0.04584511+0.23723667j]]
mat12=
[[ 1.82769508e-10+2.83327528e-12j  1.93887623e-10+1.61457153e-10j
  -4.04775990e-01+4.66634738e-01j]
 [-4.04775991e-01+4.66634739e-01j  2.53606192e-10+2.33358111e-10j
   1.67378764e-10+1.60664648e-10j]
 [ 7.62306329e-

In [432]:
eig0 = Compute_H(k_mesh4,k_Rpoint,Energy4,G_R,basis4,a=10.8,units='Hartree',details=False)
print('\n H2 \n')
Ham2, eig2 = Compute_H2_test(k_mesh4,k_Rpoint,Energy4,E_ext[:-10],G_R,basis4,basis_ext[:-10],a=10.8,units='Hartree',details=False)

Calculating bands: DONE

 H2 

Calculating bands: DONE


In [433]:
Ham2_qe, eig2_qe = Compute_H2_test(k_mesh4,k_Rpoint,Energy4,E_ext[:-10],G_R,basis4_qe,basis_ext[:-10],a=10.8,units='Hartree',details=False)

Calculating bands: DONE


In [434]:
path = '/home/mjocic/Desktop/plot_tmp'
Edft = E[20:41,17:21]#*2*13.605698066
k2=[0.0]
d=0
for i in range(len(k_mesh4)-1):
    d1 = k_mesh4[i+1]-k_mesh4[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)
x=[];y_kp=[];y_dft=[];y_kp2=[];y_kp2_qe=[]
FILE_kp = open( path+'/plotband_kp.dat','w')

for n in range(eig2.shape[1]):
    for k_i in range(len(k_mesh4)):
        x.append(k2[k_i])
        y_kp.append(eig0[k_i][n].real);
        y_kp2.append(eig2[k_i][n].real)
        y_kp2_qe.append(eig2_qe[k_i][n].real)
        y_dft.append(Edft[k_i][n])
        FILE_kp.write("{} {} {} {} {}\n".format (k2[k_i],eig0[k_i][n].real,eig2[k_i][n].real,Edft[k_i][n],eig2_qe[k_i][n].real))
        #FILE_dft.write("{} {}\n".format (k2[k_i],E[k_i][n+17].real))
    FILE_kp.write("\n")
    #FILE_dft.write("\n")
FILE_kp.close()
#FILE_dft.close()

In [435]:
import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_kp,label='$k \cdot p$')
plt.plot(x,y_kp2,label='$k \cdot p 2$')
#plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
#plt.ylim(y_dft[0],y_dft[1500])
#plt.ylim(-0.2,0.3)
plt.xticks([k2[0],k2[10],k2[19]],('X','R','M'))
plt.ylabel('E(k) [Hartree]')
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


In [458]:
def Compute_Perturbation_test(G_vec,C_terms,ext_basis,E_terms,E_ext,kkr,m,n):
        new_basis = np.vstack((ext_basis,C_terms))
        dim = len(ext_basis)
        #print('now calc for [{},{}]'.format(m,n))
        cgc = 0
        Eavg = (E_terms[m]+E_terms[n])/2
        for l in range(len(E_ext)):
            #if abs(E_ext[l]-Eavg)<1e-10: 
                #cont_count +=1
                #print('continues<',cont_count);
                #continue
            delta_E1 =1/(Eavg - E_ext[l])
            #if abs(delta_E1)>1e-1:
                #print('continues>');
                #continue
            kpi = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis,dim+m,l))
            kpj = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis,l,dim+n))
            cgc += kpi*kpj*delta_E1
        #print(cgc)
        #if np.absolute(cgc)<1e-10: return 0.0
        #else:
        return cgc
def Compute_KdP2(k_mesh,k_point,E_k,E_ext,G_space,C_terms,ext_basis,a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    H=[]
    H_eig=[]
    dim = len(C_terms)
    for k_i in range(len(k_mesh)):
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                KdP2 = Compute_Perturbation_test(G_space,C_terms,ext_basis,E_k,E_ext,kkR,m,n)
                H_i[m][n] = tpiba4*KdP2
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of KdP2_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating KdP2: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

def Compute_KdP1(k_mesh,k_point,G_space,C_terms,a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    H=[]
    H_eig=[]
    dim = len(C_terms)
    for k_i in range(len(k_mesh)):
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                KdP = Compute_Matrix_Element(G_space,C_terms,m,n)
                H_i[m][n] = tpiba2*kkR.dot(KdP)
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of KdP_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating KdP: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]
def Compute_H0(k_mesh,k_point,E_k,a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    H=[]
    H_eig=[]
    dim = len(E_k)
    for k_i in range(len(k_mesh)):
        kkR2 = k_mesh[k_i].dot(k_mesh[k_i])-k_point.dot(k_point)
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(dim):
                H_i[m][m] = E_k[m] + 0.5*tpiba2*kkR2
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of KdP_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating KdP: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

In [476]:
#old_order=np.array([C_R[17],basis_g25p[0],basis_g25p[1],basis_g25p[2]])
#new_order=np.array([C_R[17],basis_g25p[1],basis_g25p[2],basis_g25p[0]])
k_mesh4 = k_points
kdp2,    kdp2_eig    = Compute_KdP2(k_mesh4,k_Rpoint,E_R[17:21],E_ext[:-2],G_R,old_order,basis_ext[:-2],a=10.8)
kdp2_no, kdp2_eig_no = Compute_KdP2(k_mesh4,k_Rpoint,E_R[17:21],E_ext[:-3],G_R,new_order,basis_ext[:-2],a=10.8)
kdp2_qe, kdp2_eig_qe = Compute_KdP2(k_mesh4,k_Rpoint,E_R[17:21],E_ext,G_R,C_R[17:21],basis_ext,a=10.8)

Calculating KdP2: DONE
Calculating KdP2: DONE
Calculating KdP2: DONE


In [477]:
kdp1,    kdp1_eig    = Compute_KdP1(k_mesh4,k_Rpoint,G_R,old_order,a=10.8)
kdp1_no, kdp1_eig_no = Compute_KdP1(k_mesh4,k_Rpoint,G_R,new_order,a=10.8)
kdp1_qe, kdp1_eig_qe = Compute_KdP1(k_mesh4,k_Rpoint,G_R,C_R[17:21],a=10.8)

Calculating KdP: DONE
Calculating KdP: DONE
Calculating KdP: DONE


In [478]:
h0 , h0_eig = Compute_H0(k_mesh4,k_Rpoint,E_R[17:21],a=10.8)

Calculating KdP: DONE


In [450]:
s=19
print('{}\n'.format(kdp2_eig[s],kdp2[s]))
print('{}\n'.format(kdp2_eig_no[s],kdp2_no[s]))
print('{}\n'.format(kdp2_eig_qe[s],kdp2_qe[s]))

[-0.06397611-2.65918517e-19j -0.01016578-1.24681148e-18j
 -0.01184997-8.87518268e-20j -0.01184997-3.99161330e-19j]

[-0.06397611-2.65918517e-19j -0.01016578+5.64277078e-19j
 -0.01184997-9.93062585e-19j -0.01184997+4.28784994e-19j]

[-0.06397611-2.65918517e-19j -0.01016578-6.71846094e-19j
 -0.01184997+1.31802483e-18j -0.01184997-2.12498434e-19j]



In [448]:
s=19
print('{}\n'.format(kdp1_eig[s],kdp1[s]))
print('{}\n'.format(kdp1_eig_no[s],kdp1_no[s]))
print('{}\n'.format(kdp1_eig_qe[s],kdp1_qe[s]))

[-0.01793146-3.08830537e-18j  0.17024017+1.00271912e-17j
  0.07615435+6.02186245e-19j  0.07615435+2.86726100e-18j]

[-0.01793146-5.66095366e-18j  0.17024017-1.19983195e-18j
  0.07615435+1.19613179e-17j  0.07615435+1.81746728e-19j]

[-0.01793146+3.37413394e-18j  0.17024017+1.05036458e-17j
  0.07615435-5.58416500e-19j  0.07615435+5.58416486e-19j]



In [479]:
H_kp1 = h0 + kdp1 ;      eig_kp1    = np.linalg.eigvalsh(H_kp1)
H_kp1_no = h0 + kdp1_no; eig_kp1_no = np.linalg.eigvalsh(H_kp1_no)
H_kp1_qe = h0 + kdp1_qe; eig_kp1_qe = np.linalg.eigvalsh(H_kp1_qe)

In [480]:
H_kp2 = h0+kdp1+kdp2 ;         eig_kp2    = np.linalg.eigvalsh(H_kp2)
H_kp2_no = h0+kdp1_no+kdp2_no; eig_kp2_no = np.linalg.eigvalsh(H_kp2_no)
H_kp2_qe = h0+kdp1_qe+kdp2_qe; eig_kp2_qe = np.linalg.eigvalsh(H_kp2_qe)

In [488]:
path = '/home/mjocic/Desktop/plot_tmp'
units = 2*13.605698066
Edft = E[:,17:21] * units
eig_kp=eig_kp1 * units
eig_kp_no=eig_kp2_no *units
eig_kp_qe=eig_kp2_qe *units
k2=[0.0]
d=0
for i in range(len(k_mesh4)-1):
    d1 = k_mesh4[i+1]-k_mesh4[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)
x=[];y_kp=[];y_dft=[];y_kp2=[];y_kp2_qe=[]
FILE_kp = open( path+'/plotband_kp.dat','w')

for n in range(eig_kp.shape[1]):
    for k_i in range(len(k_mesh4)):
        x.append(k2[k_i])
        y_kp.append(eig_kp[k_i][n].real);
        y_kp2.append(eig_kp_no[k_i][n].real)
        y_kp2_qe.append(eig_kp_qe[k_i][n].real)
        y_dft.append(Edft[k_i][n])
        FILE_kp.write("{} {} {} {} {}\n".format (k2[k_i],eig_kp[k_i][n].real,eig_kp_no[k_i][n].real,Edft[k_i][n],eig_kp_qe[k_i][n].real))
        #FILE_dft.write("{} {}\n".format (k2[k_i],E[k_i][n+17].real))
    FILE_kp.write("\n")
    #FILE_dft.write("\n")
FILE_kp.close()
#FILE_dft.close()

In [489]:
print(k_mesh4[0], k_mesh4[10],k_mesh4[20],k_mesh4[30],k_mesh4[40])

[0. 0. 0.] [0.5 0.5 0. ] [0.  0.5 0. ] [0.5 0.5 0.5] [0.5 0.5 0. ]


In [456]:
import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_kp,label='$k \cdot p$')
plt.plot(x,y_kp2,label='$k \cdot p _{no}$')
plt.plot(x,y_kp2_qe,label='$k \cdot p_{qe}$')
plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
#plt.ylim(y_dft[0],y_dft[1500])
#plt.ylim(-0.2,0.3)
plt.xticks([k2[0],k2[10],k2[19]],('X','R','M'))
plt.ylabel('E(k) [Hartree]')
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


In [1]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_0=1
k_points= Gather_K(path)
k_0point = k_points[k_0-1]

G_0 = Gather_G(path,k_0)
C_0 = Gather_C(path,k_0)
E = Gather_E(path=path)
E_0 = E[0]
symmetry_list = Gather_Symmetries(path)

In [2]:
deg_u,deg_e = Gather_Degeneracies(E_0,functions=True)
#deg_h = Gather_Degeneracies(H[k_R-1])
for i in range(len(deg_e)):
    print(deg_e[i])
print(deg_e.shape)
print(deg_u)

['-0.5219816635323075' '2' 'index: 0-1']
['-0.512722483108768' '3' 'index: 2-4']
['-0.4552462868853928' '1' 'index: 5']
['-0.4187110503390926' '2' 'index: 6-7']
['-0.1596956348539142' '1' 'index: 8']
['-0.0744733583783985' '3' 'index: 9-11']
['0.01833436990801816' '3' 'index: 12-14']
['0.02217366076438526' '3' 'index: 15-17']
['0.2016914498130925' '1' 'index: 18']
['0.2397948734687009' '2' 'index: 19-20']
['0.2713086767092451' '3' 'index: 21-23']
['0.3361363767597791' '3' 'index: 24-26']
['0.3896321802764791' '3' 'index: 27-29']
['0.4656449194598513' '3' 'index: 30-32']
['0.478587866283999' '1' 'index: 33']
['0.4970503487772222' '2' 'index: 34-35']
['0.5547532449233975' '3' 'index: 36-38']
['0.576134192308696' '3' 'index: 39-41']
['0.5775369761532605' '2' 'index: 42-43']
['0.5880129687742724' '1' 'index: 44']
['0.6194644532604825' '3' 'index: 45-47']
['0.6386228291451255' '3' 'index: 48-50']
['0.641922374211531' '1' 'index: 51']
['0.647014915692707' '1' 'index: 52']
['0.698535903035598

In [6]:
Deg_char=[]
for uk in deg_u:
    kstart=uk[0]
    kstop=uk[-1]
    char=Gather_Charachters(kstart,kstop,symmetry_list,G_0,C_0,k_0point,details=True)
    Deg_char.append(np.array(char[:,0],dtype=int))

Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (1.9999999999999907+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (1.9999999999999916+7.659477715474502e-18j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-3.3306690738754696e-15-1.4100849964006936e-16j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-2.6645352591003757e-15-1.7172470686925827e-16j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (-0.9999999999999956+1.1102230246251565e-16j)

Now performing for inversion
charachter =  (1.9999999999999893-8.816269432807327e-18j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (1.9999999999999916-5.2036783692032865e-17j)

Now performing for inv.  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-2.9976021664879227e-15+1.0423367749740726e-16j)

Now performing for inv. 180 deg rotation 

charachter =  (1.27675647831893e-15+1.6653345369377348e-16j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (2.9999999999999885+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (-0.9999999999999939+1.4729723371841298e-16j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (0.9999999999999969-5.828670879282072e-16j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-0.999999999999997-3.793147193648133e-16j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (1.817990202823694e-15+4.926614671774132e-15j)

Now performing for inversion
charachter =  (-2.9999999999999862-6.088283235067514e-16j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (0.9999999999999973-3.3306670390111627e-16j)

Now performing for inv.  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-0.9999999999999962+1.02695629777

charachter =  (0.999999999999995-1.8741639655336736e-16j)

Now performing for inv. 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (-1.2739809207573671e-14-3.885780586188048e-16j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (1.0000000000000109+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (1.0000000000000107-9.073278933997437e-19j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-1.00000000000001+4.879315548988023e-17j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-1.0000000000000107+2.2819728051051186e-18j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (1.0000000000000104-1.9024331344509353e-17j)

Now performing for inversion
charachter =  (-1.0000000000000107+9.063987789325648e-19j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (-1.0000000000000118-6.93889371973

charachter =  (0.9999999999999984+1.7629327833194785e-16j)

Now performing for inv. 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (-1.7763568394002505e-15+2.609024107869118e-15j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (3.000000000000003+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (-1.0000000000000013+5.2397193419355e-16j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (1.0000000000000009+6.210310043996969e-16j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-1.0000000000000004-5.631526177652675e-17j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (6.217248937900877e-15+3.774758283725532e-15j)

Now performing for inversion
charachter =  (3.0000000000000036+5.2205075759005886e-17j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (-1.0000000000000013-5.5509245407

charachter =  (-0.9999999999999997+5.630912403020716e-15j)

Now performing for inv. 180 deg rotation - cart. axis [1,1,0]
charachter =  (1.0000000000000007+1.5683101760572712e-16j)

Now performing for inv. 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (-1.3877787807814457e-16-4.3021142204224816e-15j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (0.999999999999991+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (0.999999999999992-2.9474018109199146e-17j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-0.9999999999999911+2.7530823405367896e-16j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-0.999999999999991+1.3978558672348423e-17j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (0.9999999999999923-2.88347242315596e-16j)

Now performing for inversion
charachter =  (-0.9999999999999912+8.4981715447

charachter =  (-2.0000000000000027+5.551126951808542e-17j)

Now performing for inv.  90 deg rotation - cart. axis [0,0,-1]
charachter =  (3.885780586188048e-15-5.846866726904064e-17j)

Now performing for inv. 180 deg rotation - cart. axis [1,1,0]
charachter =  (3.7192471324942744e-15-3.9935098036362414e-17j)

Now performing for inv. 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (1.0000000000000004-3.4416913763379853e-15j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (3.000000000000006+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (-0.9999999999999982-2.443330502231798e-16j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (1.000000000000002+9.992007221626409e-16j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-1.0000000000000047+2.1891132417401924e-16j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter 

charachter =  (-0.9999999999999998-2.2000772891490113e-16j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (3.00000000000001+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (-0.9999999999999981-5.0575414146990664e-17j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-1.000000000000002+3.191891195797325e-15j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (1.0000000000000018+6.731467115653226e-17j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (8.881784197001252e-16-2.275957200481571e-15j)

Now performing for inversion
charachter =  (-3.0000000000000027-6.498542319581154e-17j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (1.0000000000000007-4.857243202789597e-16j)

Now performing for inv.  90 deg rotation - cart. axis [0,0,-1]
charachter =  (1.0000000000000047-3.3584246494910985

charachter =  (0.9999999999382393-2.6177618941357453e-12j)
Done
Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
charachter =  (2.9999999999999942+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]
charachter =  (-0.9999999997561396-6.247664560930354e-17j)

Now performing for  90 deg rotation - cart. axis [0,0,-1]
charachter =  (0.9999999999365983+6.167997224082455e-10j)

Now performing for 180 deg rotation - cart. axis [1,1,0]
charachter =  (-1.0000000000984541+1.1129121391657297e-16j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]
charachter =  (3.1644881159920146e-10+3.3912028740701317e-10j)

Now performing for inversion
charachter =  (-2.9999999985273247-1.9556189814472144e-16j)

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
charachter =  (0.9999999990046154-3.1918929494202237e-16j)

Now performing for inv.  90 deg rotation - cart. axis [0,0,-1]
charachter =  (-0.9999999998209933-5.063633956

In [7]:
for i in range(len(Deg_char)):
    print(deg_u[i], Deg_char[i])

[0, 1] [ 2  2  0  0 -1  2  2  0  0 -1]
[2, 3, 4] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[5] [1 1 1 1 1 1 1 1 1 1]
[6, 7] [ 2  2  0  0 -1  2  2  0  0 -1]
[8] [1 1 1 1 1 1 1 1 1 1]
[9, 10, 11] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[12, 13, 14] [ 3 -1 -1  1  0 -3  1  1 -1  0]
[15, 16, 17] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[18] [1 1 1 1 1 1 1 1 1 1]
[19, 20] [ 2  2  0  0 -1  2  2  0  0 -1]
[21, 22, 23] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[24, 25, 26] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[27, 28, 29] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[30, 31, 32] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[33] [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[34, 35] [ 2  2  0  0 -1  2  2  0  0 -1]
[36, 37, 38] [ 3 -1 -1  1  0 -3  1  1 -1  0]
[39, 40, 41] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[42, 43] [ 2  2  0  0 -1  2  2  0  0 -1]
[44] [1 1 1 1 1 1 1 1 1 1]
[45, 46, 47] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[48, 49, 50] [ 3 -1  1 -1  0  3 -1  1 -1  0]
[51] [ 1  1 -1 -1  1  1  1 -1 -1  1]
[52] [1 1 1 1 1 1 1 1 1 1]
[53, 54] [ 2  2  0  0 -1  2  2  0  0 -1]
[55, 56, 57] [ 

In [59]:
char_g1 = np.array([1,1,1,1,1,1,1,1,1,1])
char_g2 = np.array([1,1,-1,-1,1,1,1,-1,-1,1])
char_g12= np.array([2,2,0,0,-1,2,2,0,0,-1])
char_g15= np.array([3,-1,1,-1,0,-3,1,-1,1,0])
char_g25= np.array([3,-1,-1,1,0,-3,1,1,-1,0])

char_g1p= np.array([1, 1, 1, 1, 1,-1,-1,-1,-1,-1])
char_g2p= np.array([1, 1,-1,-1, 1,-1,-1, 1, 1,-1])
char_g12p=np.array([2, 2, 0, 0,-1,-2,-2, 0, 0, 1])
char_g15p=np.array([3,-1, 1,-1, 0, 3,-1, 1,-1, 0])
char_g25p=np.array([3,-1,-1, 1, 0, 3,-1,-1, 1, 0])

Oh_sym = [[char_g1,"$\Gamma_1$"],[char_g2,"$\Gamma_2$"],
          [char_g12,"$\Gamma_{12}$"],
          [char_g15,"$\Gamma_{15}$"],
          [char_g25,"$\Gamma_{25}$"]]
Oh_symp = [[char_g1p,"$\Gamma^'_1$"],[char_g2p,"$\Gamma^'_2$"],
          [char_g12p,"$\Gamma^'_{12}$"],
          [char_g15p,"$\Gamma^'_{15}$"],
          [char_g25p,"$\Gamma^'_{25}$"]]

Nk = [1,3,6,6,8,1,3,6,6,8]

ir_rep=[]
for i in range(len(Deg_char)):
    for char_g in Oh_sym:
        c=char_coef(char_g[0],Deg_char[i],Nk)/48
        if (c)==1:
              ir_rep.append(char_g[1])
    for char_g in Oh_symp:
        c=char_coef(char_g[0],Deg_char[i],Nk)/48
        if (c)==1:
              ir_rep.append(char_g[1])

for i in range(len(ir_rep)):
    print(deg_u[i],ir_rep[i],Deg_char[i])

NameError: name 'Deg_char' is not defined

In [35]:
k_start=21
k_stop=23
matrices=Gather_Matrices(k_start,k_stop,symmetry_list,G_0,C_0,k_0point)

In [17]:
for i in range(len(matrices)):
    print('\n{}\n{}'.format(matrices[i].operation,np.round(matrices[i].matrix.real)))


identity
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [ 0. -0.  1.]]

180 deg rotation - cart. axis [0,0,1]
[[ 1. -0.  1.]
 [-0. -1.  0.]
 [ 1.  0. -1.]]

180 deg rotation - cart. axis [0,1,0]
[[-1.  0. -0.]
 [ 0.  0. -1.]
 [-0. -1. -0.]]

180 deg rotation - cart. axis [1,0,0]
[[-1. -0. -0.]
 [-0. -0.  1.]
 [-0.  1. -0.]]

180 deg rotation - cart. axis [1,1,0]
[[-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0.  1.]]

180 deg rotation - cart. axis [1,-1,0]
[[-1.  0. -0.]
 [ 0.  1. -0.]
 [-0. -0. -1.]]

 90 deg rotation - cart. axis [0,0,-1]
[[ 1.  0.  0.]
 [-0.  0.  1.]
 [ 0. -1.  0.]]

 90 deg rotation - cart. axis [0,0,1]
[[ 1. -0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]]

180 deg rotation - cart. axis [1,0,1]
[[ 1. -1. -0.]
 [-1. -1.  0.]
 [-0.  0. -1.]]

180 deg rotation - cart. axis [-1,0,1]
[[-1.  0.  0.]
 [ 0. -0.  1.]
 [ 0.  1.  0.]]

 90 deg rotation - cart. axis [0,1,0]
[[ 0.  0.  0.]
 [-0.  1. -1.]
 [-1. -0.  0.]]

 90 deg rotation - cart. axis [0,-1,0]
[[ 0. -0. -1.]
 [ 0.  1. -0.]
 [ 0. -1.  0.]]

180 

In [36]:
path2="/home/mjocic/data_paradox"
G_25p = Gather_Symmetries(path2,"G25p_t2g.xml")
r = Compute_small_r(matrices,G_25p)
Unitary_transforms = Gather_Unitary_Transforms(r,matrices,G_25p)

u_11 = Unitary_transforms[0,0]
u_11i= nplin.inv(u_11)

/home/mjocic/Post_processing/CsPbBr3/CsPbBr3_collect_wf/post_procesing_tools.py:384: ComplexWarning: Casting complex values to real discards the imaginary part
  dim = int(round(sym1[0].char))


In [37]:
u_11a = np.transpose(np.conjugate(u_11))
norm = np.trace(u_11a.dot(u_11))/len(u_11)
u_11 = u_11/np.sqrt(norm)
u_11a= u_11a/np.sqrt(norm)

In [38]:
print(u_11)
print(u_11a)

[[ 0.91303716+2.02463250e-16j -0.39265435+2.52646874e-02j
   0.09892277-4.19724818e-02j]
 [ 0.18680427+4.43698890e-02j  0.31822854-1.44396023e-01j
  -0.68354602-6.11376066e-01j]
 [ 0.09920925+3.45913498e-01j  0.18586968+8.29769308e-01j
  -0.27701045+2.65898369e-01j]]
[[ 0.91303716-2.02463250e-16j  0.18680427-4.43698890e-02j
   0.09920925-3.45913498e-01j]
 [-0.39265435-2.52646874e-02j  0.31822854+1.44396023e-01j
   0.18586968-8.29769308e-01j]
 [ 0.09892277+4.19724818e-02j -0.68354602+6.11376066e-01j
  -0.27701045-2.65898369e-01j]]


In [39]:
basis_qe=C_0[k_start:k_stop+1:1]
basis_g25p = u_11.T.dot(basis_qe)

In [51]:
basis4 = np.array([C_0[33],basis_g25p[1],basis_g25p[2],basis_g25p[0]])
print(basis4.shape)
dim = len(basis4)
for m in range(dim):
    for n in range(dim):
        print('[{},{}] = \n{}'.format(m,n,Compute_Matrix_Element(G_0,basis4,m,n)))

(4, 15275)
[0,0] = 
[ 1.03982420e-10+4.46938012e-21j -4.51288087e-11-3.03730381e-25j
  6.61542517e-11+1.64143653e-24j]
[0,1] = 
[-9.49375833e-02+1.68747844e-01j -5.40420051e-11-6.39520187e-13j
 -1.73709926e-11-6.99741470e-11j]
[0,2] = 
[-3.27821410e-11-4.43726483e-11j -9.49375833e-02+1.68747844e-01j
  4.26155043e-11+2.98573872e-11j]
[0,3] = 
[ 2.17595967e-11-1.43711344e-10j -3.46244201e-11+2.63519097e-11j
 -9.49375832e-02+1.68747844e-01j]
[1,0] = 
[-9.49375833e-02-1.68747844e-01j -5.40420050e-11+6.39520120e-13j
 -1.73709926e-11+6.99741471e-11j]
[1,1] = 
[-6.63350006e-11-4.68001181e-20j -8.57182443e-12-6.12113653e-21j
 -1.27545848e-10+1.18791279e-20j]
[1,2] = 
[-2.41107899e-11-5.40488949e-11j  1.57560846e-11-1.56009246e-11j
 -1.33568150e-10+5.20018144e-11j]
[1,3] = 
[-8.73489284e-11+3.38283232e-11j -4.72950776e-11-3.78353644e-11j
 -7.52713181e-11+4.03557184e-11j]
[2,0] = 
[-3.27821411e-11+4.43726482e-11j -9.49375833e-02-1.68747844e-01j
  4.26155043e-11-2.98573872e-11j]
[2,1] = 
[-2.4110

In [68]:
char_red = char_g15*char_g25p*char_g2p
print(char_red)
for char_g in Oh_sym:
    c=char_coef(char_g[0],char_red,Nk)/48
    if (c)!=0:
          print(char_g[1],c)
for char_g in Oh_symp:
    c=char_coef(char_g[0],char_red,Nk)/48
    if (c)!=0:
          print(char_g[1],c)

[9 1 1 1 0 9 1 1 1 0]
$\Gamma_1$ 1.0
$\Gamma_{12}$ 1.0
$\Gamma^'_{15}$ 1.0
$\Gamma^'_{25}$ 1.0


In [9]:
from post_procesing_tools import *
path_so = '/home/mjocic/data_paradox/nbnd_60_spinorbit'
E_rdft = Gather_E(path_so,units='Electronvolt')
path = '/home/mjocic/data_paradox/nbnd_60'
E_dft = Gather_E(path,units='Electronvolt')
k_mesh4=Gather_K(path)

In [11]:
path = '/home/mjocic/Desktop/plot_tmp'
k2=[0.0]
d=0
for i in range(len(k_mesh4)-1):
    d1 = k_mesh4[i+1]-k_mesh4[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)
x=[];#y_kp=[];
y_dft=[]; y_rdft=[];
#y_kp2=[];y_kp2_qe=[]
FILE_kp = open( path+'/plotband_dft_vs_rdft_ev.dat','w')

for n in range(Edft.shape[1]):
    for k_i in range(len(k_mesh4)):
        x.append(k2[k_i])
        #y_kp.append(eig0[k_i][n].real);
        #y_kp2.append(eig2[k_i][n].real)
        #y_kp2_qe.append(eig2_qe[k_i][n].real)
        y_dft.append(E_dft[k_i][n])
        y_rdft.append(E_rdft[k_i][n])
        FILE_kp.write("{} {} {}\n".format (k2[k_i],E_dft[k_i][n],E_rdft[k_i][n]))
        #FILE_dft.write("{} {}\n".format (k2[k_i],E[k_i][n+17].real))
    FILE_kp.write("\n")
    #FILE_dft.write("\n")
FILE_kp.close()
#FILE_dft.close()

In [7]:
import numpy as np
import re as re

def Gather_C_so(path,k_point,details=False,form='stack'):    
    FILE_up = open (path+'/wfc{}_1.xml'.format(k_point),"r")
    FILE_dn = open (path+'/wfc{}_2.xml'.format(k_point),"r")
    nbnd=0
    igwx=0
    WF_U=[]
    WF_D=[]
    if details:
        print("Now performing for k-point={}:\n".format(k_point))
    for line in FILE_up:
        if 'INFO' in line:
            found_nbnd = re.search('nbnd="(.+?)"',line)
            if found_nbnd:
                nbnd = int(found_nbnd.group(1))
        if 'igwx' in line:
            found_igwx=re.search('igwx="(.+?)"',line)
            if found_igwx:
                igwx = int(found_igwx.group(1))
        for n in range(nbnd):
            wf_n=[]
            if '<evc.{} type='.format(n+1) in line:
                for i in range(igwx):
                    nextLine=next(FILE_up)
                    A = (nextLine.replace("\n","")).split(",")
                    C = complex(float(A[0]),float(A[1]))

                    wf_n.append(C)
                WF_U.append(wf_n)
    for line in FILE_dn:
        if 'Info' in line:
            found_nbnd = re.search('nbnd="(.+?)"',line)
            if found_nbnd:
                nbnd = int(found_nbnd.group(1))
        if 'igwx' in line:
            found_igwx=re.search('igwx="(.+?)"',line)
            if found_igwx:
                igwx = int(found_igwx.group(1))
        for n in range(nbnd):
            wf_n=[]
            if '<evc.{} type='.format(n+1) in line:
                for i in range(igwx):
                    nextLine=next(FILE_dn)
                    A = (nextLine.replace("\n","")).split(",")
                    C = complex(float(A[0]),float(A[1]))

                    wf_n.append(C)
                WF_D.append(wf_n)
    if details:
        print("\tReading wave-fuctions: DONE")
    WF_U=np.array(WF_U)
    WF_D=np.array(WF_D)
    FILE_up.close()
    FILE_dn.close()
    if form == 'stack':
        return np.array([WF_U,WF_D])
    if form == 'interweave':
        shape0 = WF_U.shape[0] + WF_D.shape[0]
        shape1 = WF_U.shape[1]
        WF = np.empty((shape0,shape1), dtype=WF_U.dtype)
        WF[0::2] = WF_U; WF[1::2] = WF_D
        return WF

def Gather_G_so(path,k_point,details=False):
    FILE = open (path+'/grid{}.xml'.format(k_point),"r")
    size=0
    GV=[]
    if details:
        print("Gathering G-vectors for k-point={}:\n".format(k_point))
    for line in FILE:
        if '<GRID type=' in line:
            size = re.search('size="(.+?)"',line)
            if size:
                size = int(int(size.group(1))/3)
            for i in range(size):
                nextLine=next(FILE)
                g = (nextLine.replace("\n","")).split()
                G_i = np.array([int(g[0]),int(g[1]),int(g[2])])
                GV.append(G_i)
    if details:
        print("\tReading G-vectors: DONE")
    GV=np.array(GV)
    return GV

In [278]:
path_so="/home/mjocic/data_paradox/nbnd_60_sp"
C_Rso = Gather_C_so(path_so,31,form='stack')
G_Rso = Gather_G_so(path_so,31)
E_Rso = Gather_E(path_so)[30]

In [279]:
print(C_Rso.shape)

(2, 60, 16524)


In [3]:
def Gather_Degeneracies_so(E_list,functions=False):
    deg_e=[]
    deg_u=[]
    index=0
    while index+4<E_list.shape[0]:
        if abs(E_list[index]-E_list[index+1])<1e-4:
            index += 1
            if abs(E_list[index]-E_list[index+1])<1e-4:
                index += 1
                if abs(E_list[index]-E_list[index+1])<1e-4:
                    index += 1
                    deg_e.append([E_list[index],4,'index: {}-{}'.format(index-3,index)])
                    deg_u.append([index-3,index-2,index-1,index])
                else:
                    deg_e.append([E_list[index],3,'index: {}-{}'.format(index-2,index)])
                    deg_u.append([index-2,index-1,index])
            else:
                deg_e.append([E_list[index],2,'index: {}-{}'.format(index-1,index)])
                deg_u.append([index-1,index])
        else:
            deg_e.append([E_list[index],1,'index: {}'.format(index)])
            deg_u.append([index])
        index +=1
    if index == E_list.shape[0] - 2:
        deg_e.append([E_list[index],2,'index: {}-{}'.format(index,index+1)])
        deg_u.append([index,index+1])
    if functions==False:
        return np.array(deg_e)
    else:
        return (deg_u),np.array(deg_e)

In [20]:
E_Rso = Gather_E(path_so)[0]
deg_u,deg_e = Gather_Degeneracies_so(E_Rso,functions=True)
#deg_h = Gather_Degeneracies(H[k_R-1])
for i in range(len(deg_e)):
    print(deg_e[i])
print(deg_e.shape)
print(deg_u)

['-0.6477342394555555' '2' 'index: 0-1']
['-0.5302637388299315' '4' 'index: 2-5']
['-0.4407118567460173' '4' 'index: 6-9']
['-0.4341730908260967' '2' 'index: 10-11']
['-0.4040516369834171' '2' 'index: 12-13']
['-0.3750888356998454' '4' 'index: 14-17']
['-0.2041670297256312' '2' 'index: 18-19']
['-0.1442081398239956' '4' 'index: 20-23']
['-0.1200614464362699' '2' 'index: 24-25']
['-0.030776388185724' '2' 'index: 26-27']
['-0.01936083594087517' '4' 'index: 28-31']
['0.05304209390084765' '2' 'index: 32-33']
['0.0584156504739805' '4' 'index: 34-37']
['0.0667181040890033' '2' 'index: 38-39']
['0.071801018068145' '4' 'index: 40-43']
['0.2451307528005072' '2' 'index: 44-45']
['0.2870301939860093' '4' 'index: 46-49']
['0.3074974453975586' '2' 'index: 50-51']
['0.3140191621954269' '4' 'index: 52-55']
(19, 3)
[[0, 1], [2, 3, 4, 5], [6, 7, 8, 9], [10, 11], [12, 13], [14, 15, 16, 17], [18, 19], [20, 21, 22, 23], [24, 25], [26, 27], [28, 29, 30, 31], [32, 33], [34, 35, 36, 37], [38, 39], [40, 41, 4

In [263]:
print(E_Rso)

[-0.64773424 -0.64773424 -0.53026374 -0.53026374 -0.53026374 -0.53026374
 -0.44071186 -0.44071186 -0.44071186 -0.44071186 -0.43417309 -0.43417309
 -0.40405164 -0.40405164 -0.37508884 -0.37508884 -0.37508884 -0.37508884
 -0.20416703 -0.20416703 -0.14420814 -0.14420814 -0.14420814 -0.14420814
 -0.12006145 -0.12006145 -0.03077639 -0.03077639 -0.01936084 -0.01936084
 -0.01936084 -0.01936084  0.05304209  0.05304209  0.05841565  0.05841565
  0.05841565  0.05841565  0.0667181   0.0667181   0.07180102  0.07180102
  0.07180102  0.07180102  0.24513075  0.24513075  0.28703019  0.28703019
  0.28703019  0.28703019  0.30749745  0.30749745  0.31401916  0.31401916
  0.31401916  0.31401916  0.31598019  0.31598019  0.37394636  0.37394636]


In [70]:
np.sqrt(2)

1.4142135623730951

In [4]:
def Gather_Symmetries_double(path,file_name="index.xml"):
    full_path=path+"/"+file_name
    FILE = open (full_path,"r")
    n_sym=0
    inv_sym = False
    sym_list=[]
    sym_list_d=[]
    sigma_list=[]
    sigma_list_d=[]
    for line in FILE:
        if '<symmops' in line:
            n_sym = re.search('nsym="(.+?)"',line)
            if n_sym:
                n_sym = int(n_sym.group(1))
    for i in range(1,n_sym+1):
        FILE = open (full_path,"r")
        for line in FILE:
            if '<info.{} name'.format(i) in line:
                #Single-group
                sym_name = re.search('name="(.+?)"',line)
                sym_name = str(sym_name.group(1))
                print(sym_name)
                nextLine=next(FILE)
                mat_type = re.search('type="integer"',nextLine)
                nextLine=next(FILE)
                r = nextLine.split()
                R1 = [int(r[0]),int(r[1]),int(r[2])]
                nextLine=next(FILE)
                r = nextLine.split()
                R2 = [int(r[0]),int(r[1]),int(r[2])]
                nextLine=next(FILE)
                r = nextLine.split()
                R3 = [int(r[0]),int(r[1]),int(r[2])]
                mat = np.array([R1,R2,R3])
                R = Symmetry(mat,sym_name)
                Rd= Symmetry(mat,sym_name+"+d")
                sym_list.append(R)
                sym_list_d.append(Rd)
                #Double-group:
                nextLine=next(FILE)
                mat_type = re.search('type="complex"',nextLine)
                nextLine=next(FILE)
                s = nextLine.split()
                print(s)
                S1 = [eval(s[0],{'sqrt':np.sqrt}),eval(s[1],{'sqrt':np.sqrt})]
                nextLine=next(FILE)
                s = nextLine.split()
                S2 = [eval(s[0],{'sqrt':np.sqrt}),eval(s[1],{'sqrt':np.sqrt})]
                sigma = np.array([S1,S2],dtype=complex)
                print(sigma)
                S = Symmetry(sigma,sym_name)
                Sd = Symmetry(-sigma,sym_name+"+d")
                sigma_list.append(S)
                sigma_list_d.append(Sd)
    sym_list.extend(sym_list_d)
    sigma_list.extend(sigma_list_d)
    return sym_list, sigma_list

In [11]:
sym_path="/home/mjocic/Post_processing/Symmetry_groups"
file_name="Oh_double2.xml"
symmetries_double,sig_double=Gather_Symmetries_double(sym_path,file_name)

In [12]:
for i in range(len(symmetries_double)):
    print("[{}] {}".format(i,symmetries_double[i].operation))

[0] identity
[1] 180 deg rotation - cart. axis [0,0,1]
[2] 180 deg rotation - cart. axis [0,1,0]
[3] 180 deg rotation - cart. axis [1,0,0]
[4] 180 deg rotation - cart. axis [1,1,0]
[5] 180 deg rotation - cart. axis [1,-1,0]
[6]  90 deg rotation - cart. axis [0,0,-1]
[7]  90 deg rotation - cart. axis [0,0,1]
[8] 180 deg rotation - cart. axis [1,0,1]
[9] 180 deg rotation - cart. axis [-1,0,1]
[10]  90 deg rotation - cart. axis [0,1,0]
[11]  90 deg rotation - cart. axis [0,-1,0]
[12] 180 deg rotation - cart. axis [0,1,1]
[13] 180 deg rotation - cart. axis [0,1,-1]
[14]  90 deg rotation - cart. axis [-1,0,0]
[15]  90 deg rotation - cart. axis [1,0,0]
[16] 120 deg rotation - cart. axis [-1,-1,-1]
[17] 120 deg rotation - cart. axis [-1,1,1]
[18] 120 deg rotation - cart. axis [1,1,-1]
[19] 120 deg rotation - cart. axis [1,-1,1]
[20] 120 deg rotation - cart. axis [1,1,1]
[21] 120 deg rotation - cart. axis [-1,1,-1]
[22] 120 deg rotation - cart. axis [1,-1,-1]
[23] 120 deg rotation - cart. axis

In [100]:
def Gather_Charachters_double(k_start,k_stop,symmetry_list,sigma_list,G_R,C_Rup,C_Rdn,k_Rpoint,details=False):
    charachters=[]
    dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
    #dim_G = int(dim_G)
    if details: print('Dimension of 3x3 square matrix is:', dim_G)
    M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
    if details: print('Creating G-matrix')
    for i in range(len(G_R)):
        n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
        M[n1,n2,n3] = i
    if details: print('G-matrix: Done')
    for s in [0,1,16,8,6,48,64,54,24,25,40,32,38,72,88,78]:
        sym_name=symmetry_list[s].operation
        sym_matrix=symmetry_list[s].matrix
        sigma_matrix=sigma_list[s].matrix
        if details: 
            print('\nNow performing for '+ sym_name)
            print('\nSymm.Op. matrix:\n{}\nSpin.Op. matrix:\n{}'.format(sym_matrix,sigma_matrix))
        aa = sigma_matrix[0,0]
        ab = sigma_matrix[0,1]
        ba = sigma_matrix[1,0]
        bb = sigma_matrix[1,1]
        char=0
        for k in range(k_start,k_stop+1):
            RC_Rup = np.zeros(len(G_R),dtype=complex)
            RC_Rdn = np.zeros(len(G_R),dtype=complex)
            
            for i in range(len(G_R)):
                g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
                n1,n2,n3=int(g_i[0]),int(g_i[1]),int(g_i[2])
                index = M[n1,n2,n3]
                RC_Rup[i] = C_Rup[k,index]
                RC_Rdn[i] = C_Rdn[k,index]
            
            up_up = np.vdot(RC_Rup,C_Rup[k])*aa
            up_dn = np.vdot(RC_Rup,C_Rdn[k])*ab
            dn_up = np.vdot(RC_Rdn,C_Rup[k])*ba
            dn_dn = np.vdot(RC_Rdn,C_Rdn[k])*bb
            
            char += up_up + up_dn + dn_up + dn_dn
        if details: print('charachter = ',round(char,3))
        charachters.append([round(char.real,3), sym_name])
    if details: print('Done')
    return np.array(charachters)

### Example spin-orbit

In [6]:
from post_procesing_tools import *
from post_procesing_tools_double import *

path_so="/home/mjocic/data_paradox/nbnd_60_sp"
C_Rso = Gather_C_so(path_so,1,form='stack')
G_Rso = Gather_G_so(path_so,1)
k_path = Gather_K(path=path_so)

In [8]:
k_Rpoint=k_path[30]
print(k_Rpoint)

[0.5 0.5 0.5]


In [13]:
sym_path="/home/mjocic/Post_processing/Symmetry_groups"
file_name="Oh_double2.xml"
symmetries_double,sig_double=Gather_Symmetries_double(sym_path,file_name)

In [14]:
for i in range(len(symmetries_double)):
    print("[{}] {}".format(i,symmetries_double[i].operation))

[0] identity
[1] 180 deg rotation - cart. axis [0,0,1]
[2] 180 deg rotation - cart. axis [0,1,0]
[3] 180 deg rotation - cart. axis [1,0,0]
[4] 180 deg rotation - cart. axis [1,1,0]
[5] 180 deg rotation - cart. axis [1,-1,0]
[6]  90 deg rotation - cart. axis [0,0,-1]
[7]  90 deg rotation - cart. axis [0,0,1]
[8] 180 deg rotation - cart. axis [1,0,1]
[9] 180 deg rotation - cart. axis [-1,0,1]
[10]  90 deg rotation - cart. axis [0,1,0]
[11]  90 deg rotation - cart. axis [0,-1,0]
[12] 180 deg rotation - cart. axis [0,1,1]
[13] 180 deg rotation - cart. axis [0,1,-1]
[14]  90 deg rotation - cart. axis [-1,0,0]
[15]  90 deg rotation - cart. axis [1,0,0]
[16] 120 deg rotation - cart. axis [-1,-1,-1]
[17] 120 deg rotation - cart. axis [-1,1,1]
[18] 120 deg rotation - cart. axis [1,1,-1]
[19] 120 deg rotation - cart. axis [1,-1,1]
[20] 120 deg rotation - cart. axis [1,1,1]
[21] 120 deg rotation - cart. axis [-1,1,-1]
[22] 120 deg rotation - cart. axis [1,-1,-1]
[23] 120 deg rotation - cart. axis

In [16]:
E_Rso = Gather_E(path_so)[30]
deg_u,deg_e = Gather_Degeneracies_so(E_Rso,functions=True)
#deg_h = Gather_Degeneracies(H[k_R-1])
for i in range(len(deg_e)):
    print(deg_e[i])
print(deg_e.shape)
print(deg_u)

['-0.647630843559593' '2' 'index: 0-1']
['-0.529567260245753' '4' 'index: 2-5']
['-0.4359062970494437' '4' 'index: 6-9']
['-0.4342778212932227' '2' 'index: 10-11']
['-0.3897186637014939' '2' 'index: 12-13']
['-0.3890529579196721' '4' 'index: 14-17']
['-0.2050278805781854' '2' 'index: 18-19']
['-0.1937909556202406' '2' 'index: 20-21']
['-0.1457939457138693' '4' 'index: 22-25']
['0.03053687862751674' '4' 'index: 26-29']
['0.03772941675642138' '2' 'index: 30-31']
['0.05790407791378965' '4' 'index: 32-35']
['0.07175885161207506' '2' 'index: 36-37']
['0.08338743720841985' '4' 'index: 38-41']
['0.1407395186570235' '2' 'index: 42-43']
['0.1457064839616963' '2' 'index: 44-45']
['0.2058323389733331' '4' 'index: 46-49']
['0.3010372870545936' '2' 'index: 50-51']
['0.3225223002350731' '2' 'index: 52-53']
['0.3305242273692915' '4' 'index: 54-57']
['0.3743166803580156' '2' 'index: 58-59']
(21, 3)
[[0, 1], [2, 3, 4, 5], [6, 7, 8, 9], [10, 11], [12, 13], [14, 15, 16, 17], [18, 19], [20, 21], [22, 23, 

In [15]:
k_start=10
k_stop=11

charachters_double=Gather_Charachters_double(
    k_start,k_stop,symmetries_double,sig_double,
    G_Rso,C_Rso[0],C_Rso[1],k_Rpoint,details=True)

Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity

Symm.Op. matrix:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Spin.Op. matrix:
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
charachter =  (2+0j)

Now performing for 180 deg rotation - cart. axis [0,0,1]

Symm.Op. matrix:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.-1.j  0.+0.j]
 [ 0.+0.j  0.+1.j]]
charachter =  (-0-0j)

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]

Symm.Op. matrix:
[[0 1 0]
 [0 0 1]
 [1 0 0]]
Spin.Op. matrix:
[[ 0.5+0.5j  0.5+0.5j]
 [-0.5+0.5j  0.5-0.5j]]
charachter =  (1-0j)

Now performing for 180 deg rotation - cart. axis [1,0,1]

Symm.Op. matrix:
[[ 0  0  1]
 [ 0 -1  0]
 [ 1  0  0]]
Spin.Op. matrix:
[[0.-0.70710678j 0.-0.70710678j]
 [0.-0.70710678j 0.+0.70710678j]]
charachter =  0j

Now performing for  90 deg rotation - cart. axis [0,0,-1]

Symm.Op. matrix:
[[ 0  1  0]
 [-1  0  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[0.70710678+0.70710678j 0.        +0.j        ]
 [0.

In [91]:
print(G_Rso.shape)
print(C_Rso.shape)
print(k_Rpoint)
print(np.amin(G_Rso))

(16524, 3)
(2, 60, 16524)
[0.5 0.5 0.5]
-16


In [101]:
E_Rso1 = Gather_E(path_so)[0]
E_Rso2 = Gather_E(path_so)[30]
deg_u1,deg_e1 = Gather_Degeneracies_so(E_Rso1,functions=True)
deg_u2,deg_e2 = Gather_Degeneracies_so(E_Rso2,functions=True)
#deg_h = Gather_Degeneracies(H[k_R-1])
for i in range(len(deg_e)):
    print(deg_e1[i][2],deg_e2[i][2])
print(deg_e.shape)
print(deg_u)

index: 0-1 index: 0-1
index: 2-5 index: 2-5
index: 6-9 index: 6-9
index: 10-11 index: 10-11
index: 12-13 index: 12-13
index: 14-17 index: 14-17
index: 18-19 index: 18-19
index: 20-23 index: 20-21
index: 24-25 index: 22-25
index: 26-27 index: 26-29
index: 28-31 index: 30-31
index: 32-33 index: 32-35
index: 34-37 index: 36-37
index: 38-39 index: 38-41
index: 40-43 index: 42-43
index: 44-45 index: 44-45
index: 46-49 index: 46-49
index: 50-51 index: 50-51
index: 52-55 index: 52-53
(19, 3)
[[0, 1], [2, 3, 4, 5], [6, 7, 8, 9], [10, 11], [12, 13], [14, 15, 16, 17], [18, 19], [20, 21, 22, 23], [24, 25], [26, 27], [28, 29, 30, 31], [32, 33], [34, 35, 36, 37], [38, 39], [40, 41, 42, 43], [44, 45], [46, 47, 48, 49], [50, 51], [52, 53, 54, 55]]


In [263]:
Deg_char=[]
for uk in deg_u2:
    print(uk)
    k_start=uk[0]
    k_stop=uk[-1]
    char=Gather_Charachters_double(
    k_start,k_stop,symmetries_double,sig_double,
    G_Rso,C_Rso[0],C_Rso[1],k_Rpoint,details=False)
    Deg_char.append(np.array(char[:,0],dtype=float))

[0, 1]
[2, 3, 4, 5]
[6, 7, 8, 9]
[10, 11]
[12, 13]
[14, 15, 16, 17]
[18, 19]
[20, 21]
[22, 23, 24, 25]
[26, 27, 28, 29]
[30, 31]
[32, 33, 34, 35]
[36, 37]
[38, 39, 40, 41]
[42, 43]
[44, 45]
[46, 47, 48, 49]
[50, 51]
[52, 53]
[54, 55, 56, 57]
[58, 59]


In [266]:
for i in range(len(Deg_char)):
    print("{}".format(Deg_char[i]))

[ 2.    -0.     1.    -0.     1.414 -2.    -1.    -1.414  2.     0.
  1.    -0.     1.414 -2.    -1.    -1.414]
[ 4.  0. -1.  0.  0. -4.  1. -0. -4. -0.  1.  0. -0.  4. -1.  0.]
[ 4.  0. -1. -0.  0. -4.  1. -0. -4.  0.  1. -0. -0.  4. -1.  0.]
[ 2.    -0.     1.    -0.     1.414 -2.    -1.    -1.414 -2.    -0.
 -1.    -0.    -1.414  2.     1.     1.414]
[ 2.     0.     1.    -0.    -1.414 -2.    -1.     1.414  2.     0.
  1.    -0.    -1.414 -2.    -1.     1.414]
[ 4. -0. -1.  0. -0. -4.  1.  0.  4. -0. -1. -0. -0. -4.  1.  0.]
[ 2.    -0.     1.     0.     1.414 -2.    -1.    -1.414 -2.     0.
 -1.    -0.    -1.414  2.     1.     1.414]
[ 2.    -0.     1.    -0.    -1.414 -2.    -1.     1.414 -2.     0.
 -1.    -0.     1.414  2.     1.    -1.414]
[ 4.  0. -1.  0.  0. -4.  1. -0. -4.  0.  1. -0. -0.  4. -1.  0.]
[ 4. -0. -1. -0. -0. -4.  1.  0. -4. -0.  1.  0. -0.  4. -1.  0.]
[ 2.     0.     1.    -0.     1.414 -2.    -1.    -1.414 -2.    -0.
 -1.     0.    -1.414  2.     1.     1.414

In [268]:
char_g6p = np.array([2,0,1,0,1.414,-2,-1,-1.414,2,0,1,0,1.414,-2,-1,-1.414])
char_g7p = np.array([2,0,1,0,-1.414,-2,-1,1.414,2,0,1,0,-1.414,-2,-1,1.414])
char_g6m= np.array([2,0,1,0,1.414,-2,-1,-1.414,-2,0,-1,0,-1.414,2,1,1.414])
char_g7m= np.array([2,0,1,0,-1.414,-2,-1,1.414,-2,0,-1,0,1.414,2,1,-1.414])

char_g8p= np.array([4,0,-1,0,0,-4,1,0,4,0,-1,0,0,-4,1,0])
char_g8m= np.array([4,0,-1,0,0,-4,1,0,-4,0,1,0,0,4,-1,0])

Nk_double = [1,6,8,12,6,1,8,6,1,6,8,12,6,1,8,6]

Oh_symm = [[char_g6m,"$\Gamma_6^-$"],[char_g7m,"$\Gamma_7^-$"],
          [char_g8m,"$\Gamma_8^-$"]]
Oh_symp = [[char_g6p,"$\Gamma_6^+$"],[char_g7p,"$\Gamma_7^+$"],
          [char_g8p,"$\Gamma_8^+$"]]



ir_rep=[]

for i in range(len(Deg_char)):
    for char_g in Oh_symm:
        c=char_coef(char_g[0],Deg_char[i],Nk_double)/96
        #print(c)
        if abs(c-1.0)<1e-3:
            #print("{}\n{}".format(c,char_g))
            ir_rep.append(char_g[1])
    for char_g in Oh_symp:
        c=char_coef(char_g[0],Deg_char[i],Nk_double)/96
        #print(c)
        if abs(c-1.0)<1e-3:
            #print("{}\n{}".format(c,char_g))
            ir_rep.append(char_g[1])

for i in range(len(ir_rep)):
    print("{} irrep: {}".format(deg_e2[i][2],ir_rep[i],Deg_char[i]))

index: 0-1 irrep: $\Gamma_6^+$
index: 2-5 irrep: $\Gamma_8^-$
index: 6-9 irrep: $\Gamma_8^-$
index: 10-11 irrep: $\Gamma_6^-$
index: 12-13 irrep: $\Gamma_7^+$
index: 14-17 irrep: $\Gamma_8^+$
index: 18-19 irrep: $\Gamma_6^-$
index: 20-21 irrep: $\Gamma_7^-$
index: 22-25 irrep: $\Gamma_8^-$
index: 26-29 irrep: $\Gamma_8^-$
index: 30-31 irrep: $\Gamma_6^-$
index: 32-35 irrep: $\Gamma_8^-$
index: 36-37 irrep: $\Gamma_7^-$
index: 38-41 irrep: $\Gamma_8^-$
index: 42-43 irrep: $\Gamma_7^-$
index: 44-45 irrep: $\Gamma_7^+$
index: 46-49 irrep: $\Gamma_8^+$
index: 50-51 irrep: $\Gamma_6^+$
index: 52-53 irrep: $\Gamma_7^+$
index: 54-57 irrep: $\Gamma_8^+$


21

In [159]:
def Check_Orthonormality_spin(WF_up,WF_dn, tol=1e-13,details=False):
    # check orthonormality
    nbnd = len(WF_up)
    Ort_MAT=np.zeros((nbnd,nbnd),dtype=complex)
    for i in range(nbnd):
        for j in range(i,nbnd):
            A_up=np.array(WF_up[i])
            A_dn=np.array(WF_dn[i])
            B_up=np.array(WF_up[j])
            B_dn=np.array(WF_dn[j])
            Ort_MAT[i][j]=np.vdot(A_up,B_up)+np.vdot(A_dn,B_dn)
            Ort_MAT[j][i]=Ort_MAT[i][j]
    if details:
        print("\tVector product is: DONE")
    
    orthonormal=True
    #print("\nDiagonal elements:\n")
    tol_global=tol
    index = [0,0]
    for i in range(nbnd):
        if abs(abs(Ort_MAT[i][i])-1.0)<tol:
            pass#print (1)
        else:
            if details:
                print ("\n\tMatrix element [{},{}] is:{}".format(i,i,Ort_MAT[i][i]))
            orthonormal=False
            ort_new=False
            tol_new=tol
            while ort_new==False:
                    tol_new=tol_new*10
                    if tol_global<tol_new: tol_global=tol_new; index=[i,i]
                    if abs(abs(Ort_MAT[i][i])-1.0)<tol_new:
                        if details:
                            print("\tTolerance for {}".format(tol_new))
                        ort_new=True
    #print("\nOff-diagonal elements:\n")
    for i in range(nbnd):
        for j in range(i+1,nbnd):
            if abs(Ort_MAT[i][j])<tol:
                pass#print(0)
            else:
                if details:
                    print ("\n\tMatrix element: {},{} is:{}".format(i,j,abs(Ort_MAT[i][j])))
                orthonormal=False
                ort_new=False
                tol_new=tol
                while ort_new==False:
                    tol_new=tol_new*10
                    if tol_global<tol_new: tol_global=tol_new; index=[i,j]
                    if abs(Ort_MAT[i][j])<tol_new:
                        if details: 
                            print("\tTolerance for {}".format(tol_new))
                        ort_new=True                    
    print('\nOrthonormality {} for tolerance {}'.format(orthonormal,tol))
    if not orthonormal:
        print('Orthonormal for tolerance {} in index {}\n'.format(tol_global,index))

In [238]:
def Gather_Charachters_Dtest(k_start,k_stop,symmetry_list,sigma_list,
                             G_R,C_Rup,C_Rdn,k_Rpoint,details=False):
    charachters=[]
    dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
    #dim_G = int(dim_G)
    if details: print('Dimension of 3x3 square matrix is:', dim_G)
    M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
    if details: print('Creating G-matrix')
    for i in range(len(G_R)):
        n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
        M[n1,n2,n3] = i
    if details: print('G-matrix: Done')
    for s in [24]:
        sym_name=symmetry_list[s].operation
        sym_matrix=symmetry_list[s].matrix
        sigma_matrix=sigma_list[s].matrix
        if details: 
            print('\nNow performing for '+ sym_name)
            print('\nSymm.Op. matrix:\n{}\nSpin.Op. matrix:\n{}'.format(sym_matrix,sigma_matrix))
        aa = sigma_matrix[0,0]
        ab = sigma_matrix[0,1]
        ba = sigma_matrix[1,0]
        bb = sigma_matrix[1,1]
        dim = 1+k_stop-k_start
        print(dim)
        mat = np.zeros((dim,dim),dtype=complex)
        print(mat)
        for k1 in range(k_start,k_stop+1):
            for k2 in range(k_start,k_stop+1):
                RC_Rup = np.zeros(len(G_R),dtype=complex)
                RC_Rdn = np.zeros(len(G_R),dtype=complex)
            
                for i in range(len(G_R)):
                    g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
                    n1,n2,n3=int(g_i[0]),int(g_i[1]),int(g_i[2])
                    index = M[n1,n2,n3]
                    RC_Rup[i] = C_Rup[k1,index]
                    RC_Rdn[i] = C_Rdn[k1,index]
            
                up_up = np.vdot(RC_Rup,C_Rup[k2])*aa
                up_dn = np.vdot(RC_Rup,C_Rdn[k2])*ab
                dn_up = np.vdot(RC_Rdn,C_Rup[k2])*ba
                dn_dn = np.vdot(RC_Rdn,C_Rdn[k2])*bb
            #print("\nk={},{}".format(k1,k2))
            #print("sum=",up_up + up_dn + dn_up + dn_dn)
            #print("uu={}".format(np.vdot(RC_Rup,C_Rup[k2])))
            #print("ud={}".format(np.vdot(RC_Rup,C_Rdn[k2])))
            #print("du={}".format(np.vdot(RC_Rdn,C_Rup[k2])))
            #print("dd={}".format(np.vdot(RC_Rdn,C_Rdn[k2])))
                mat[k1-k_start,k2-k_start]= up_up + up_dn + dn_up + dn_dn
        print(mat)
        char=np.matrix.trace(mat)
        if details: print('charachter = ',round(char,3))
        charachters.append([round(char.real,3), sym_name])
    if details: print('Done')
    return np.array(charachters),RC_Rup,RC_Rdn

In [260]:
from post_procesing_tools import *

#path_so="/home/mjocic/data_paradox/nbnd_60_sp"
path_so_new="/home/mjocic/data_paradox/nbnd_60_sp_new"
#path_so_new_grid="/home/mjocic/data_paradox/nbnd_60_sp_new/new_grid"
C_Rso = Gather_C_so(path_so_new,31,form='stack')
G_Rso = Gather_G_so(path_so_new,31)
k_path = Gather_K(path=path_so)
k_Rpoint=k_path[30]

In [261]:
k_start=18
k_stop =19
char_test,RC_Rup,RC_Rdn=Gather_Charachters_Dtest(k_start,k_stop,symmetries_double,
                                   sig_double,G_Rso,C_Rso[0],C_Rso[1],
                                   k_Rpoint,details=True)

Dimension of 3x3 square matrix is: 33
Creating G-matrix
G-matrix: Done

Now performing for inversion

Symm.Op. matrix:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
Spin.Op. matrix:
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
2
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[-1.00000000e+00+1.51919577e-17j -1.04430353e-14+2.57849297e-14j]
 [-1.04499742e-14-2.54518628e-14j -1.00000000e+00-1.16480292e-16j]]
charachter =  (-2-0j)
Done


In [254]:
print(k_Rpoint)

[0.5 0.5 0.5]


In [208]:
print(C_Rso[0,19]/RC_Rup)

[ 1.        +0.00000000e+00j -0.99999998-1.08381156e-08j
 -0.99999999-1.11528986e-08j ... -1.00000228-2.39440429e-07j
 -1.00000154+3.47888394e-06j -1.00000406+9.40933652e-07j]


In [200]:
unit = np.array([1,1,1])
RC_up2=RC_Rup*np.array(np.exp(-1j*2*(k_Rpoint+G_Rso).dot(unit)))

In [201]:
print(RC_up2)

[-7.48322322e-02-2.88332666e-02j  7.55862766e-02-1.19580613e-01j
  5.18995022e-03-7.59103631e-02j ...  7.38395424e-07-6.49247426e-07j
  1.23665009e-06+1.98282558e-07j -1.26462610e-06-1.07032002e-06j]


In [202]:
np.vdot(RC_up2,RC_Rup)

(-0.0060207980927366594+0.05854028799161134j)

In [1]:
def Compute_Perturbation_spin(G_vec,C_terms_up,C_terms_dn,
                              ext_basis_up,ext_basis_dn,
                              E_terms,E_ext,kkr,m,n):
        new_basis_up = np.vstack((ext_basis_up,C_terms_up))
        new_basis_dn = np.vstack((ext_basis_dn,C_terms_dn))
        dim = len(ext_basis_up)
        cgc = 0
        Eavg = (E_terms[m]+E_terms[n])/2
        for l in range(len(E_ext)):
            delta_E1 =1/(Eavg - E_ext[l])
            ml_up = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis_up,dim+m,l))
            ml_dn = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis_dn,dim+m,l))
            ln_up = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis_up,l,dim+n))
            ln_dn = np.dot(kkr,Compute_Matrix_Element(G_vec,new_basis_dn,l,dim+n))
            kpi = ml_up + ml_dn
            kpj = ln_up + ln_dn
            cgc += kpi*kpj*delta_E1
        return cgc
    
def Compute_KdP2_spin(k_mesh,k_point,E_k,E_ext,G_space,
                      C_terms_up,C_terms_dn,
                      ext_basis_up,ext_basis_dn,
                      a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    H=[]
    H_eig=[]
    dim = len(C_terms_up)
    for k_i in range(len(k_mesh)):
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(dim):
            for n in range(dim):
                KdP2 = Compute_Perturbation_spin(G_space,C_terms_up,C_terms_dn,
                                                 ext_basis_up,ext_basis_dn,
                                                 E_k,E_ext,kkR,m,n)
                H_i[m][n] = tpiba4*KdP2
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of KdP2_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating KdP2: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

def Compute_KdP1_spin(k_mesh,k_point,G_space,
                      C_terms_up,C_terms_dn,a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    H=[]
    H_eig=[]
    dim = len(C_terms_up)
    for k_i in range(len(k_mesh)):
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(dim):
            for n in range(dim):
                KdP_up = Compute_Matrix_Element(G_space,C_terms_up,m,n)
                KdP_dn = Compute_Matrix_Element(G_space,C_terms_dn,m,n)
                H_i[m][n] = tpiba2*kkR.dot(KdP_up+KdP_dn)
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of KdP_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating KdP1: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

def Compute_H0(k_mesh,k_point,E_k,a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    H=[]
    H_eig=[]
    dim = len(E_k)
    for k_i in range(len(k_mesh)):
        kkR2 = k_mesh[k_i].dot(k_mesh[k_i])-k_point.dot(k_point)
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(dim):
                H_i[m][m] = E_k[m] + 0.5*tpiba2*kkR2
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of KdP_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating H0: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

In [2]:
from post_procesing_tools_double import *

path_so="/home/mjocic/data_paradox/nbnd_60_sp"

C_Rso = Gather_C_so(path_so,31,form='stack')
G_Rso = Gather_G_so(path_so,31)

E_Rso = Gather_E(path_so)[30]

k_path = Gather_K(path=path_so)
k_Rpoint=k_path[30]

In [3]:
basis8 = np.array([C_Rso[0,42:50],C_Rso[1,42:50]])
basis_ext_up = np.vstack((C_Rso[0,:42],C_Rso[0,50:]))
basis_ext_dn = np.vstack((C_Rso[1,:42],C_Rso[1,50:]))
Energy8 = E_Rso[42:50]
E_ext = np.append(E_Rso[:42],E_Rso[50:])

In [6]:
k_mesh = k_path[20:40]
h0 , h0_eig = Compute_H0(k_mesh,k_Rpoint,Energy8,a=10.8)
kdp1, kdp1_eig = Compute_KdP1_spin(k_mesh,k_Rpoint,
                                   G_Rso,basis8[0],basis8[1],a=10.8)
kdp2, kdp2_eig = Compute_KdP2_spin(k_mesh,k_Rpoint,Energy8,E_ext,
                                   G_Rso,basis8[0],basis8[1],
                                   basis_ext_up,basis_ext_dn,a=10.8)
relkin, relkin_eig = Compute_KdP_RelKin(k_mesh,k_Rpoint,
                                   G_Rso,basis8[0],basis8[1],a=10.8)

Calculating RelKin: DONE


In [7]:
H_1 = h0 + kdp1 ;        eig_h1 = np.linalg.eigvalsh(H_1)
H_2 = H_1 + kdp2 ;       eig_h2 = np.linalg.eigvalsh(H_2)
H_3 = H_2 + relkin;      eig_h3 = np.linalg.eigvalsh(H_3)

In [19]:
k2=[0.0]
d=0
k_mesh4=k_mesh
for i in range(len(k_mesh4)-1):
    d1 = k_mesh4[i+1]-k_mesh4[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)

#path = '/home/mjocic/Desktop/plot_tmp'
units = 1*2*13.605698066
Edft = Gather_E(path_so)*units

eig_0=eig_h3 * units
eig_1=eig_h1 *units
eig_2=eig_h2 *units
x=[];y_0=[];y_dft=[];y_1=[];y_2=[]
#FILE_kp = open( path+'/plotband_h012.dat','w')

for n in range(eig_1.shape[1]):
    for k_i in range(len(k_mesh4)):
        x.append(k2[k_i])
        y_0.append(eig_0[k_i][n].real);
        y_1.append(eig_1[k_i][n].real)
        y_2.append(eig_2[k_i][n].real)
        y_dft.append(Edft[k_i][n])
        #FILE_kp.write("{} {} {} {} {}\n".format (k2[k_i],eig_0[k_i][n].real,eig_1[k_i][n].real,eig_2[k_i][n].real,Edft[k_i][n]))
    #FILE_kp.write("\n")
    #FILE_dft.write("\n")
#FILE_kp.close()
#FILE_dft.close()

In [17]:
print(eig_1.shape)
print(k2.shape)
print(k_mesh4.shape)

(20, 8)
(20,)
(20, 3)


In [25]:
import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_0,label='$H_2+relkin$')
#plt.plot(x,y_1,label='$H_1$')
plt.plot(x,y_2,label='$H_2$')
#plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
#plt.ylim(y_dft[0],y_dft[1500])
#plt.ylim(-0.2,0.3)
#plt.xticks([k2[0],k2[10],k2[19]],('X','R','M'))
plt.ylabel('E(k) [Hartree]')
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


In [5]:
def Compute_Matrix_Element_kpp2(kkr,G_vec,C_terms,m,n):
    A = np.array(C_terms[m])
    B = np.array(C_terms[n])
    Gx=G_vec[:,0]
    Gy=G_vec[:,1]
    Gz=G_vec[:,2]
    G2=Gx*Gx + Gy*Gy + Gz*Gz
    G = [np.vdot(A,G2*Gx*B),np.vdot(A,G2*Gy*B),np.vdot(A,G2*Gz*B)]
    return np.dot(kkr,G)

def Compute_Matrix_Element_kp2(kkr,G_vec,C_terms,m,n):
    A = np.array(C_terms[m])
    B = np.array(C_terms[n])
    Gx=G_vec[:,0]
    Gy=G_vec[:,1]
    Gz=G_vec[:,2]
    G = np.array([Gx,Gy,Gz])
    kp2=0
    for i in range(3):
        for j in range(3):
            kp2+=kkr[i]*kkr[j]*np.vdot(A,G[i]*Gx[j]*B)
    return kp2
def Compute_Matrix_Element_k2p2(kkr,G_vec,C_terms,m,n):
    A = np.array(C_terms[m])
    B = np.array(C_terms[n])
    Gx=G_vec[:,0]
    Gy=G_vec[:,1]
    Gz=G_vec[:,2]
    k2 = kkr*kkr
    G2 = [np.vdot(A,Gx*Gx*B),np.vdot(A,Gy*Gy*B),np.vdot(A,Gz*Gz*B)]
    return np.dot(k2,G2)
    
    
    
def Compute_KdP_RelKin(k_mesh,k_point,G_space,
                      C_terms_up,C_terms_dn,a,units='Hartree',details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    tpiba4=tpiba2**2
    H=[]
    H_eig=[]
    c = 137
    dim = len(C_terms_up)
    for k_i in range(len(k_mesh)):
        kkR = k_mesh[k_i]-k_point
        kkR2 = np.dot(kkR,kkR)
        H_i = np.zeros((dim,dim),dtype=complex)
        for m in range(dim):
            for n in range(dim):
                kpp2_up = Compute_Matrix_Element_kpp2(kkR,G_space,C_terms_up,m,n)
                kpp2_dn = Compute_Matrix_Element_kpp2(kkR,G_space,C_terms_dn,m,n)
                kpp2 = kpp2_up + kpp2_dn
                kp2_up = Compute_Matrix_Element_kp2(kkR,G_space,C_terms_up,m,n)
                kp2_dn = Compute_Matrix_Element_kp2(kkR,G_space,C_terms_dn,m,n)
                kp2 = kp2_up + kp2_dn
                k2p2_up = Compute_Matrix_Element_k2p2(kkR,G_space,C_terms_up,m,n)
                k2p2_dn = Compute_Matrix_Element_k2p2(kkR,G_space,C_terms_dn,m,n)
                k2p2 = k2p2_up + k2p2_dn
                kp_up = Compute_Matrix_Element(G_space,C_terms_up,m,n)
                kp_dn = Compute_Matrix_Element(G_space,C_terms_dn,m,n)
                kp = np.dot(kkR,kp_up + kp_dn)
                H_i[m][n] = 4*kpp2 + 4*kp2 + 2*k2p2 + 4*kkR2*kp + kkR2*kkR2
                H_i[m][n] = -tpiba4/(8*c*c)*H_i[m][n]
        H_i_eig = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        H_eig.append(H_i_eig)
        if details:
            print("eigenvalues of RelKin_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i_eig)*uni[1])
    print('Calculating RelKin: DONE')
    return np.array(H)*uni[1], np.array(H_eig)*uni[1]

In [332]:
k_mesh = k_path[20:40]

h0 , h0_eig = Compute_H0(k_mesh,k_Rpoint,Energy8,a=10.8)
kdp1, kdp1_eig = Compute_KdP1_spin(k_mesh,k_Rpoint,
                                   G_Rso,basis8[0],basis8[1],a=10.8)
kdp2, kdp2_eig = Compute_KdP2_spin(k_mesh,k_Rpoint,Energy8,E_ext,
                                   G_Rso,basis8[0],basis8[1],
                                   basis_ext_up,basis_ext_dn,a=10.8)
relkin, relkin_eig = Compute_KdP_RelKin(k_mesh,k_Rpoint,
                                   G_Rso,basis8[0],basis8[1],a=10.8)

eigenvalues of RelKin_1 in units of Hartree
[-2.29734238e-05 -2.25645834e-05 -1.11783547e-05 -1.03117445e-05
 -8.29349429e-06 -8.10684574e-06 -5.09564588e-06 -5.03186065e-06]
eigenvalues of RelKin_2 in units of Hartree
[-2.01851617e-05 -1.99228931e-05 -9.61630944e-06 -9.05427079e-06
 -7.24849140e-06 -7.11549726e-06 -4.46635855e-06 -4.42252865e-06]
eigenvalues of RelKin_3 in units of Hartree
[-1.75498230e-05 -1.73890533e-05 -8.20582172e-06 -7.85927014e-06
 -6.26565267e-06 -6.17645862e-06 -3.86750763e-06 -3.83902297e-06]
eigenvalues of RelKin_4 in units of Hartree
[-1.50475929e-05 -1.49547002e-05 -6.92285999e-06 -6.72217159e-06
 -5.34067742e-06 -5.28516300e-06 -3.29780145e-06 -3.28053465e-06]
eigenvalues of RelKin_5 in units of Hartree
[-1.26610788e-05 -1.26114847e-05 -5.74554924e-06 -5.63840542e-06
 -4.46848944e-06 -4.43704770e-06 -2.75580697e-06 -2.74623463e-06]
eigenvalues of RelKin_6 in units of Hartree
[-1.03747971e-05 -1.03510710e-05 -4.65460277e-06 -4.60340304e-06
 -3.64328335e-06

In [341]:
from post_procesing_tools_double import *

path_so="/home/mjocic/data_paradox/nbnd_80_sp"

C_Rso = Gather_C_so(path_so,31,form='stack')
G_Rso = Gather_G_so(path_so,31)

E_Rso = Gather_E(path_so)[30]

k_path = Gather_K(path=path_so)
k_Rpoint=k_path[30]

In [342]:
h0 , h0_eig = Compute_H0(k_path,k_Rpoint,E_Rso,a=10.8)
kdp1, kdp1_eig = Compute_KdP1_spin(k_path,k_Rpoint,
                                   G_Rso,C_Rso[0],C_Rso[1],a=10.8)

Calculating KdP: DONE
Calculating KdP1: DONE


In [343]:
H_1 = h0 + kdp1 ;        eig_h1 = np.linalg.eigvalsh(H_1)

In [357]:
from post_procesing_tools_double import *

path_so="/home/mjocic/data_paradox/nbnd_240_sp"

C_Rso = Gather_C_so(path_so,31,form='stack')
G_Rso = Gather_G_so(path_so,31)

E_Rso = Gather_E(path_so)[30]

k_path = Gather_K(path=path_so)
k_Rpoint=k_path[30]

In [358]:
basis8 = np.array([C_Rso[0,42:50],C_Rso[1,42:50]])
basis_ext_up = np.vstack((C_Rso[0,:42],C_Rso[0,50:]))
basis_ext_dn = np.vstack((C_Rso[1,:42],C_Rso[1,50:]))
Energy8 = E_Rso[42:50]
E_ext = np.append(E_Rso[:42],E_Rso[50:])

In [359]:
k_mesh = k_path[20:40]
h0 , h0_eig = Compute_H0(k_mesh,k_Rpoint,Energy8,a=10.8)
kdp1, kdp1_eig = Compute_KdP1_spin(k_mesh,k_Rpoint,
                                   G_Rso,basis8[0],basis8[1],a=10.8)
kdp2, kdp2_eig = Compute_KdP2_spin(k_mesh,k_Rpoint,Energy8,E_ext,
                                   G_Rso,basis8[0],basis8[1],
                                   basis_ext_up,basis_ext_dn,a=10.8)

Calculating KdP: DONE
Calculating KdP1: DONE
Calculating KdP2: DONE


In [ ]:
H_1 = h0 + kdp1 ;               eig_h1 = np.linalg.eigvalsh(H_1)
H_2 = h0 + kdp1 + kdp2 ;        eig_h2 = np.linalg.eigvalsh(H_2)

## Irreducible representations for double groups

In [6]:
import numpy as np

def Gather_irrep2(path,file_name="index.xml"):
    full_path=path+"/"+file_name
    FILE = open (full_path,"r")
    n_sym=0
    inv_sym = False
    sigma_list=[]
    for line in FILE:
        if '<symmops' in line:
            n_sym = re.search('nsym="(.+?)"',line)
            if n_sym:
                n_sym = int(n_sym.group(1))
    for i in range(1,n_sym+1):
        FILE = open (full_path,"r")
        for line in FILE:
            if '<info.{} name'.format(i) in line:
                #Double-group:
                sym_name = re.search('name="(.+?)"',line)
                sym_name = str(sym_name.group(1))
                #print(sym_name)
                nextLine=next(FILE)
                mat_type = re.search('type="complex"',nextLine)
                nextLine=next(FILE)
                s = nextLine.split()
                #print(s)
                S1 = [eval(s[0],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[1],{'exp':np.exp,'pi':np.pi})]
                nextLine=next(FILE)
                s = nextLine.split()
                S2 = [eval(s[0],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[1],{'exp':np.exp,'pi':np.pi})]
                sigma = np.array([S1,S2],dtype=complex)
                S = Symmetry(sigma,sym_name)
                sigma_list.append(S)
    return sigma_list
def Gather_irrep4(path,file_name="index.xml"):
    full_path=path+"/"+file_name
    FILE = open (full_path,"r")
    n_sym=0
    inv_sym = False
    sigma_list=[]
    for line in FILE:
        if '<symmops' in line:
            n_sym = re.search('nsym="(.+?)"',line)
            if n_sym:
                n_sym = int(n_sym.group(1))
    for i in range(1,n_sym+1):
        FILE = open (full_path,"r")
        for line in FILE:
            if '<info.{} name'.format(i) in line:
                #Double-group:
                sym_name = re.search('name="(.+?)"',line)
                sym_name = str(sym_name.group(1))
                #print(sym_name)
                nextLine=next(FILE)
                mat_type = re.search('type="complex"',nextLine)
                nextLine=next(FILE)
                s = nextLine.split()
                #print(s)
                S1 = [eval(s[0],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[1],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[2],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[3],{'exp':np.exp,'pi':np.pi})]
                #print(S1)
                nextLine=next(FILE)
                s = nextLine.split()
                S2 = [eval(s[0],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[1],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[2],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[3],{'exp':np.exp,'pi':np.pi})]
                nextLine=next(FILE)
                s = nextLine.split()
                S3 = [eval(s[0],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[1],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[2],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[3],{'exp':np.exp,'pi':np.pi})]
                nextLine=next(FILE)
                s = nextLine.split()
                S4 = [eval(s[0],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[1],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[2],{'exp':np.exp,'pi':np.pi}),\
                      eval(s[3],{'exp':np.exp,'pi':np.pi})]
                sigma = np.array([S1,S2,S3,S4],dtype=complex)
                S = Symmetry(sigma,sym_name)
                sigma_list.append(S)
    return sigma_list

In [172]:
def Gather_Matrices_double(k_start,k_stop,
                           symmetry_list,sigma_list,
                           G_R,C_Rup,C_Rdn,k_Rpoint,
                           details=False,all_elements=False,
                          elements_list=0):
    Matrices=[]
    dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
    #dim_G = int(dim_G)
    if details: print('Dimension of 3x3x3 square matrix is:', dim_G)
    M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
    if details: print('Creating G-matrix')
    for i in range(len(G_R)):
        n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
        M[n1,n2,n3] = i
    if details: print('G-matrix: Done')
    if all_elements:
        s_list = range(len(symmetry_list))
    else:
        s_list = [0,1,4,12,14,48,52,62,24,25,28,36,38,72,76,87]
    if elements_list!=0:
        s_list = elements_list
    for s in s_list:
        sym_name=symmetry_list[s].operation
        sym_matrix=symmetry_list[s].matrix
        sigma_matrix=sigma_list[s].matrix
        if details: 
            print('\nNow performing for '+ sym_name)
            print('\nSymm.Op. matrix:\n{}\nSpin.Op. matrix:\n{}'.format(sym_matrix,sigma_matrix))
        aa = sigma_matrix[0,0]
        ab = sigma_matrix[0,1]
        ba = sigma_matrix[1,0]
        bb = sigma_matrix[1,1]
        
        dim = k_stop - k_start +1
        full_mat = np.zeros((dim,dim),dtype=complex)
        for k in range(k_start,k_stop+1):
            RC_Rup = np.zeros(len(G_R),dtype=complex)
            RC_Rdn = np.zeros(len(G_R),dtype=complex)
            
            for i in range(len(G_R)):
                g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
                n1,n2,n3=int(g_i[0]),int(g_i[1]),int(g_i[2])
                index = M[n1,n2,n3]
                RC_Rup[i] = C_Rup[k,index]
                RC_Rdn[i] = C_Rdn[k,index]
                for n in range(k_start,k_stop+1):
                    
                    up_up = np.vdot(RC_Rup,C_Rup[n])*aa
                    up_dn = np.vdot(RC_Rup,C_Rdn[n])*ab
                    dn_up = np.vdot(RC_Rdn,C_Rup[n])*ba
                    dn_dn = np.vdot(RC_Rdn,C_Rdn[n])*bb
                
                    mat_comp = up_up + up_dn + dn_up + dn_dn
                    if abs(mat_comp.real)<1e-9:
                        mat_comp = mat_comp - mat_comp.real
                    if abs(mat_comp.imag)<1e-9:
                        mat_comp = mat_comp -1j*mat_comp.imag
                    
                    full_mat[k-k_start,n-k_start] = mat_comp
        if details: print('Matrix = \n',full_mat)
        Matrices.append(Symmetry(full_mat,sym_name))
    if details: print('Done')
    return np.array(Matrices)

In [291]:
from post_procesing_tools_double import *

path_so="/home/mjocic/data_paradox/nbnd_60_sp"

C_Rso = Gather_C_so(path_so,31,form='stack')
G_Rso = Gather_G_so(path_so,31)

sym_path="/home/mjocic/Post_processing/Symmetry_groups"
file_name="Oh_double2_bilbao_order.xml"
symmetries_double,sig_double=Gather_Symmetries_double(sym_path,file_name)

In [3]:
k_index = 30
E_Rso = Gather_E(path_so)[k_index]
k_path = Gather_K(path=path_so)
k_Rpoint=k_path[k_index]

In [4]:
k_start=38
k_stop=41
Matrices=Gather_Matrices_double(
    k_start,k_stop,symmetries_double,sig_double,
    G_Rso,C_Rso[0],C_Rso[1],k_Rpoint,details=True)

Dimension of 3x3x3 square matrix is: 33
Creating G-matrix
G-matrix: Done

Now performing for identity

Symm.Op. matrix:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Spin.Op. matrix:
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Matrix = 
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

Now performing for 180 deg rotation - cart. axis [0,0,1]

Symm.Op. matrix:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.-1.j  0.+0.j]
 [ 0.+0.j  0.+1.j]]
Matrix = 
 [[ 0.        -0.04156695j  0.11846513-0.50564897j -0.36744879+0.24940295j
  -0.35905469-0.63436315j]
 [-0.11846513-0.50564897j  0.        -0.6356696j   0.44934682-0.0107295j
   0.34290188+0.08123978j]
 [ 0.36744879+0.24940295j -0.44934682-0.0107295j   0.        +0.52782126j
   0.53574969-0.18742695j]
 [ 0.35905469-0.63436315j -0.34290188+0.08123978j -0.53574969-0.18742695j
   0.        +0.14941529j]]

Now performing for 120 deg rotation - cart. axis [1,1,1]

Symm.Op. matrix:
[[0 

Matrix = 
 [[ 0.48102254+0.13005646j  0.49111938-0.42803606j  0.25673116+0.36178634j
  -0.32584053-0.15593982j]
 [ 0.1364219 -0.67833235j  0.19632452-0.16309143j -0.58339935+0.02535521j
   0.15397344-0.30233533j]
 [ 0.03871418+0.15469741j -0.05297791+0.03307206j -0.51158429-0.25543586j
  -0.78033803+0.1864835j ]
 [-0.34162088-0.36169327j  0.66135265+0.26297719j  0.28643934-0.23062656j
  -0.16576277+0.28847083j]]
Done


In [113]:
path = "/home/mjocic/Post_processing/Symmetry_groups"
file = "GM8m_Fu_GM11_bilbao_order_pyreadable.xml"
GM8p = Gather_irrep4(path,file)
file_d = "GM8m_Fu_GM11+d_bilbao_order_pyreadable.xml"
GM8p_d = Gather_irrep4(path,file_d)

In [28]:
path = "/home/mjocic/Post_processing/Symmetry_groups"
file = "GM6p_E1g_GM6_bilbao_order_pyreadable.xml"
GM6p = Gather_irrep2(path,file)
file_d = "GM6p_E1g_GM6+d_bilbao_order_pyreadable.xml"
GM6p_d = Gather_irrep2(path,file_d)

In [163]:
k_start=50
k_stop=51
Matrices=Gather_Matrices_double(
    k_start,k_stop,symmetries_double,sig_double,
    G_Rso,C_Rso[0],C_Rso[1],k_Rpoint,details=True)

Dimension of 3x3x3 square matrix is: 33
Creating G-matrix
G-matrix: Done

Now performing for identity

Symm.Op. matrix:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Spin.Op. matrix:
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Matrix = 
 [[1.00000000e+00+0.00000000e+00j 1.55431223e-15+3.88578059e-16j]
 [1.55431223e-15-3.88578059e-16j 1.00000000e+00+0.00000000e+00j]]

Now performing for 180 deg rotation - cart. axis [0,0,1]

Symm.Op. matrix:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.-1.j  0.+0.j]
 [ 0.+0.j  0.+1.j]]
Matrix = 
 [[ 2.51218805e-18+0.32369328j  6.43008124e-01-0.69409165j]
 [-6.43008124e-01-0.69409165j -2.13655456e-16-0.32369328j]]

Now performing for 120 deg rotation - cart. axis [1,1,1]

Symm.Op. matrix:
[[0 0 1]
 [1 0 0]
 [0 1 0]]
Spin.Op. matrix:
[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]
Matrix = 
 [[ 0.5       +0.21305431j  0.82680067+0.14494311j]
 [-0.82680067+0.14494311j  0.5       -0.21305431j]]

Now performing for 180 deg rotation - cart. axis [1,1,0]

Symm.Op. matrix:
[

In [30]:
matrices_gm6p=Matrices
GM6p.extend(GM6p_d)
GM6p_compact=[]
for s in [0,1,4,12,14,48,52,62,24,25,28,36,38,72,76,87]:
    GM6p_compact.append(GM6p[s])

In [31]:
print(len(matrices_gm6p))
print(len(GM6p_compact))
for i in range(len(matrices_gm6p)):
    el1 = matrices_gm6p[i]
    el2 = GM6p_compact[i]
    print('operation1: {}\noperation2: {}'.format(el1.operation,el2.operation))
    print('char1: {}\nchar2: {}\n'.format(el1.char,el2.char))

16
16
operation1: identity
operation2: identity
char1: (2.000000000000004+0j)
char2: (2+0j)

operation1: 180 deg rotation - cart. axis [0,0,1]
operation2: 180 deg rotation - cart. axis [0,0,1]
char1: 4.227229677411515e-12j
char2: 0j

operation1: 120 deg rotation - cart. axis [1,1,1]
operation2: 120 deg rotation - cart. axis [1,1,1]
char1: (0.9999999999859711-3.200273379633245e-12j)
char2: (1.0000000000000002+0j)

operation1: 180 deg rotation - cart. axis [1,1,0]
operation2: 180 deg rotation - cart. axis [1,1,0]
char1: -5.705103056641292e-12j
char2: 0j

operation1:  90 deg rotation - cart. axis [0,0,-1]
operation2:  90 deg rotation - cart. axis [0,0,-1]
char1: (1.4142135623600296-2.4875101978238945e-12j)
char2: (1.4142135623730951+0j)

operation1: identity+d
operation2: identity+d
char1: (-2.000000000000004+0j)
char2: (-2+0j)

operation1: 120 deg rotation - cart. axis [1,1,1]+d
operation2: 120 deg rotation - cart. axis [1,1,1]+d
char1: (-0.9999999999859711+3.200273379633245e-12j)
char2:

In [32]:
r = Compute_small_r(matrices_gm6p,GM6p_compact)

/home/mjocic/Post_processing/CsPbBr3/CsPbBr3_collect_wf/post_procesing_tools.py:396: ComplexWarning: Casting complex values to real discards the imaginary part
  dim = int(round(sym1[0].char))


In [38]:
Unitary_transforms = Gather_Unitary_Transforms(r,matrices_gm6p,GM6p_compact)

u_11 = Unitary_transforms[0,0]
u_11i= nplin.inv(u_11)

u_11a = np.transpose(np.conjugate(u_11))
#norm = np.trace(u_11a.dot(u_11))/len(u_11)
#u_11 = u_11/np.sqrt(norm)
#u_11a= u_11a/np.sqrt(norm)

In [41]:
print(u_11)
print(u_11i)
print(u_11a)
print(u_11.dot(u_11i))

[[ 6.49654843-0.18908004j -1.8870031 -0.56206575j]
 [ 4.24215913-6.45396783j -4.86373382-1.66426697j]]
[[ 0.1644893 -0.07761355j -0.06175666+0.03223507j]
 [-0.05135962-0.26839083j -0.16026651+0.16490377j]]
[[ 6.49654843+0.18908004j  4.24215913+6.45396783j]
 [-1.8870031 +0.56206575j -4.86373382+1.66426697j]]
[[1.00000000e+00-2.03830008e-17j 1.38777878e-17+8.32667268e-17j]
 [0.00000000e+00+2.22044605e-16j 1.00000000e+00-1.11022302e-16j]]


In [42]:
basis2_up = C_Rso[0,k_start:k_stop+1]
basis2_dn = C_Rso[1,k_start:k_stop+1]
print(basis2_up.shape)

(2, 16560)


In [60]:
basis2_gm6p_up = u_11.T.dot(basis2_up)
basis2_gm6p_dn = u_11.T.dot(basis2_dn)
basis2_gm6p_upi= u_11i.T.dot(basis2_up)
basis2_gm6p_dni= u_11i.T.dot(basis2_dn)

In [61]:
m=0
n=0
A=np.vdot(basis2_gm6p_upi[m],basis2_gm6p_up[n])
B=np.vdot(basis2_gm6p_dni[m],basis2_gm6p_dn[n])
print ("{}\n{}\n{}".format(A, B, A+B))
print(u_11.trace())

(2.485536967568365+1.9635881107193722j)
(0.11206096058157251-0.02043964955558824j)
(2.5975979281499377+1.943148461163784j)
(1.6328146039282947-1.853347004488082j)


In [ ]:
A=np.vdot(basis4_up[m],basis4_up[n])
B=np.vdot(basis4_dn[m],basis4_dn[n])
print (A, B, A+B)

In [72]:
print(len(matrices_gm6p))
print(len(GM6p_compact))
for i in range(len(matrices_gm6p)):
    el1 = matrices_gm6p[i]
    el2 = GM6p_compact[i]
    print('\n*********\noperation1: {}\noperation2: {}'.format(el1.operation,el2.operation))
    print('char1: {}\nchar2: {}\n'.format(el1.char,el2.char))
    transf_mat = (u_11i.dot(el1.matrix)).dot(u_11)
    print('mat1:\n {}\nmat2:\n {}\nmat3:\n {}'
          .format(el1.matrix,el2.matrix,transf_mat))

16
16

*********
operation1: identity
operation2: identity
char1: (2.000000000000004+0j)
char2: (2+0j)

mat1:
 [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
mat2:
 [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
mat3:
 [[ 1.00000000e+00-1.16573418e-15j -7.26849136e-16+7.63278329e-17j]
 [-8.88178420e-16-3.77475828e-15j  1.00000000e+00+1.16573418e-15j]]

*********
operation1: 180 deg rotation - cart. axis [0,0,1]
operation2: 180 deg rotation - cart. axis [0,0,1]
char1: 4.227229677411515e-12j
char2: 0j

mat1:
 [[ 0.        +0.32369328j  0.64300812-0.69409165j]
 [-0.64300812-0.69409165j  0.        -0.32369328j]]
mat2:
 [[-0.-1.j  0.+0.j]
 [ 0.+0.j  0.+1.j]]
mat3:
 [[-0.08471239-0.54029231j -0.6651522 +0.40580161j]
 [ 0.72246528+0.57838865j  0.08471239+0.54029231j]]

*********
operation1: 120 deg rotation - cart. axis [1,1,1]
operation2: 120 deg rotation - cart. axis [1,1,1]
char1: (0.9999999999859711-3.200273379633245e-12j)
char2: (1.0000000000000002+0j)

mat1:
 [[ 0.5       +0.21305431j  0.82680067+0.14494311j]


### Check if matrices really form a group:
    A,B from G => AB is from G also

In [173]:
k_start=50
k_stop=51
g6p_full=Gather_Matrices_double(
    k_start,k_stop,symmetries_double,sig_double,
    G_Rso,C_Rso[0],C_Rso[1],k_Rpoint,details=True,all_elements=True)

Dimension of 3x3x3 square matrix is: 33
Creating G-matrix
G-matrix: Done

Now performing for identity

Symm.Op. matrix:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Spin.Op. matrix:
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Matrix = 
 [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

Now performing for 180 deg rotation - cart. axis [0,0,1]

Symm.Op. matrix:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.-1.j  0.+0.j]
 [ 0.+0.j  0.+1.j]]
Matrix = 
 [[ 0.        +0.32369328j  0.64300812-0.69409165j]
 [-0.64300812-0.69409165j  0.        -0.32369328j]]

Now performing for 180 deg rotation - cart. axis [0,1,0]

Symm.Op. matrix:
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
Spin.Op. matrix:
[[ 0.+0.j -1.+0.j]
 [ 1.+0.j  0.+0.j]]
Matrix = 
 [[ 0.        +0.71828271j  0.31050028+0.6226231j ]
 [-0.31050028+0.6226231j   0.        -0.71828271j]]

Now performing for 180 deg rotation - cart. axis [1,0,0]

Symm.Op. matrix:
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
Spin.Op. matrix:
[[ 0.+0.j -0.-1.j]
 [-0.-1.j  0.+0.j]]
Matrix = 
 [[ 0.   

Matrix = 
 [[ 0.        -0.61586736j  0.70009294+0.36135476j]
 [-0.70009294+0.36135476j  0.        +0.61586736j]]

Now performing for inv. 120 deg rotation - cart. axis [1,1,1]

Symm.Op. matrix:
[[ 0  0 -1]
 [-1  0  0]
 [ 0 -1  0]]
Spin.Op. matrix:
[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]
Matrix = 
 [[ 0.5       +0.21305431j  0.82680067+0.14494311j]
 [-0.82680067+0.14494311j  0.5       -0.21305431j]]

Now performing for inv. 120 deg rotation - cart. axis [-1,1,-1]

Symm.Op. matrix:
[[ 0  0 -1]
 [ 1  0  0]
 [ 0  1  0]]
Spin.Op. matrix:
[[ 0.5+0.5j -0.5+0.5j]
 [ 0.5+0.5j  0.5-0.5j]]
Matrix = 
 [[ 0.5       +0.5052284j  -0.51630039+0.47767999j]
 [ 0.51630039+0.47767999j  0.5       -0.5052284j ]]

Now performing for inv. 120 deg rotation - cart. axis [1,-1,-1]

Symm.Op. matrix:
[[ 0  0  1]
 [ 1  0  0]
 [ 0 -1  0]]
Spin.Op. matrix:
[[ 0.5+0.5j  0.5-0.5j]
 [-0.5-0.5j  0.5-0.5j]]
Matrix = 
 [[ 0.5       -0.82892168j -0.12670773+0.21641165j]
 [ 0.12670773+0.21641165j  0.5       +0.828921

Matrix = 
 [[-0.5       +0.82892168j  0.12670773-0.21641165j]
 [-0.12670773-0.21641165j -0.5       -0.82892168j]]

Now performing for 120 deg rotation - cart. axis [-1,-1,1]+d

Symm.Op. matrix:
[[ 0  0 -1]
 [ 1  0  0]
 [ 0 -1  0]]
Spin.Op. matrix:
[[-0.5+0.5j -0.5-0.5j]
 [ 0.5-0.5j -0.5-0.5j]]
Matrix = 
 [[-0.5       -0.11063897j  0.18379255+0.83903475j]
 [-0.18379255+0.83903475j -0.5       +0.11063897j]]

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]+d

Symm.Op. matrix:
[[0 1 0]
 [0 0 1]
 [1 0 0]]
Spin.Op. matrix:
[[-0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j -0.5+0.5j]]
Matrix = 
 [[-0.5       +0.21305431j  0.82680067+0.14494311j]
 [-0.82680067+0.14494311j -0.5       -0.21305431j]]

Now performing for 120 deg rotation - cart. axis [-1,1,1]+d

Symm.Op. matrix:
[[ 0 -1  0]
 [ 0  0  1]
 [-1  0  0]]
Spin.Op. matrix:
[[-0.5+0.5j  0.5-0.5j]
 [-0.5-0.5j -0.5-0.5j]]
Matrix = 
 [[-0.5       -0.82892168j -0.12670773+0.21641165j]
 [ 0.12670773+0.21641165j -0.5       +0.82892168j]]

Now perfo

Matrix = 
 [[-0.5       -0.82892168j -0.12670773+0.21641165j]
 [ 0.12670773+0.21641165j -0.5       +0.82892168j]]

Now performing for inv. 120 deg rotation - cart. axis [1,1,-1]+d

Symm.Op. matrix:
[[ 0 -1  0]
 [ 0  0  1]
 [ 1  0  0]]
Spin.Op. matrix:
[[-0.5-0.5j  0.5+0.5j]
 [-0.5+0.5j -0.5+0.5j]]
Matrix = 
 [[-0.5       +0.11063897j -0.18379255-0.83903475j]
 [ 0.18379255-0.83903475j -0.5       -0.11063897j]]

Now performing for inv. 120 deg rotation - cart. axis [1,-1,1]+d

Symm.Op. matrix:
[[ 0  1  0]
 [ 0  0  1]
 [-1  0  0]]
Spin.Op. matrix:
[[-0.5+0.5j -0.5+0.5j]
 [ 0.5+0.5j -0.5-0.5j]]
Matrix = 
 [[-0.5       +0.5052284j  -0.51630039+0.47767999j]
 [ 0.51630039+0.47767999j -0.5       -0.5052284j ]]

Now performing for inv. 180 deg rotation - cart. axis [1,1,0]+d

Symm.Op. matrix:
[[ 0 -1  0]
 [-1  0  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.        -0.j          0.70710678+0.70710678j]
 [-0.70710678+0.70710678j -0.        -0.j        ]]
Matrix = 
 [[ 0.        -0.07241858j -0.71459732

In [104]:
print((g8p_full[4].matrix-g8p_full[4].matrix<1e-4).all())

True


In [150]:
m1 = 7
m2 = 34
print('m={}\t{}'.format(m1,g8p_full[m1].operation))
print('m={}\t{}\n'.format(m2,g8p_full[m2].operation))
prod = (g8p_full[m1].matrix).dot(g8p_full[m2].matrix)
print('product={}\n'.format(prod))
for i in range(len(g8p_full)):
    if( (abs(prod-g8p_full[i].matrix)<1e-10).all() or
        (abs(prod+g8p_full[i].matrix)<1e-10).all()):
        print('op.num={}\n{}\n{}\n'.format(i,g8p_full[i].operation,g8p_full[i].matrix))

m=7	120 deg rotation - cart. axis [-1,-1,1]
m=34	inv. 120 deg rotation - cart. axis [1,1,-1]

product=[[ 1.00000000e+00-6.47176757e-13j -2.14162021e-13+1.71179737e-12j
   3.62901376e-12+7.93920485e-13j  1.75319828e-12-1.71182513e-12j]
 [ 7.38173411e-13-1.52702850e-12j  1.00000000e+00-4.66404693e-13j
  -5.11854448e-13-3.29945793e-12j -3.06263348e-12-1.12951315e-12j]
 [ 3.26483285e-12-6.60194122e-13j -2.11820145e-12+2.53616572e-12j
   1.00000000e+00-9.85794779e-13j -4.23382163e-12+4.18035051e-12j]
 [ 1.63137559e-12+5.50040569e-12j -1.26665345e-12+3.04783976e-13j
  -2.97621650e-12-5.49985057e-12j  1.00000000e+00-3.50794394e-12j]]

op.num=0
identity
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

op.num=24
inversion
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

op.num=48
identity+d
[[-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j 

In [151]:
#m1 = 1
#m2 = 78
print('m={}\t{}'.format(m1,g6p_full[m1].operation))
print('m={}\t{}\n'.format(m2,g6p_full[m2].operation))
prod = (g6p_full[m1].matrix).dot(g6p_full[m2].matrix)
print('product={}\n'.format(prod))
for i in range(len(g6p_full)):
    if( (abs(prod-g6p_full[i].matrix)<1e-10).all() or 
        (abs(prod+g6p_full[i].matrix)<1e-10).all()):
        print('op.num={}\n{}\n{}\n'.format(i,g6p_full[i].operation,g6p_full[i].matrix))

m=7	120 deg rotation - cart. axis [-1,-1,1]
m=34	inv. 120 deg rotation - cart. axis [1,1,-1]

product=[[ 1.00000000e+00+6.93833879e-13j -4.05652944e-12+8.34887715e-13j]
 [-6.15084048e-13+1.28519417e-12j  1.00000000e+00+4.43645121e-13j]]

op.num=0
identity
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

op.num=24
inversion
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

op.num=48
identity+d
[[-1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]

op.num=72
inversion+d
[[-1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]



In [152]:
#m1 = 1
#m2 = 78
print('m={}\t{}'.format(m1,GM8p[m1].operation))
print('m={}\t{}\n'.format(m2,GM8p[m2].operation))
prod = (GM8p[m1].matrix).dot(GM8p[m2].matrix)
print('product={}\n'.format(prod))
for i in range(len(GM8p)):
    if( (abs(prod-GM8p[i].matrix)<1e-11).all() or
        (abs(prod+GM8p[i].matrix)<1e-11).all()):
        print('op.num={}\n{}\n{}\n'.format(i,GM8p[i].operation,GM8p[i].matrix))

m=7	120 deg rotation - cart. axis [-1,-1,1]
m=34	inv. 120 deg rotation - cart. axis [1,1,-1]

product=[[-1.00000000e+00+2.22409305e-16j  5.55111512e-17+1.66533454e-16j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+1.66533454e-16j -1.00000000e+00+8.36314274e-17j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -1.00000000e+00-1.38777878e-16j  1.38777878e-16-4.20375091e-17j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -1.40503659e-16-2.04125361e-17j -1.00000000e+00-5.55111512e-17j]]

op.num=0
identity
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

op.num=24
inversion
[[-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]

op.num=48
identity+d
[[-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [

In [127]:
print((abs(GM8p[4].matrix-GM8p[35].matrix)<1e-10).all())

False


In [153]:
#m1 = 1
#m2 = 78
print('m={}\t{}'.format(m1,symmetries_double[m1].operation))
print('m={}\t{}\n'.format(m2,symmetries_double[m2].operation))
prod = (symmetries_double[m1].matrix).dot(symmetries_double[m2].matrix)
prod_sig = (sig_double[m1].matrix).dot(sig_double[m2].matrix)
print('product=\n{}\n'.format(prod))
print('product sigma=\n{}\n'.format(prod_sig))
for i in range(len(GM8p)):
    if(( (abs(prod-symmetries_double[i].matrix)<1e-11).all() or
        (abs(prod+symmetries_double[i].matrix)<1e-11).all() ) and
      ( (abs(prod_sig-sig_double[i].matrix)<1e-11).all() or
        (abs(prod_sig+sig_double[i].matrix)<1e-11).all() ) ):
        print('op.num={}\n{}\n{}\n{}\n'.format(i,
        symmetries_double[i].operation,
        symmetries_double[i].matrix,
        sig_double[i].matrix))

m=7	120 deg rotation - cart. axis [-1,-1,1]
m=34	inv. 120 deg rotation - cart. axis [1,1,-1]

product=
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]

product sigma=
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

op.num=0
identity
[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

op.num=24
inversion
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

op.num=48
identity+d
[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[-1.-0.j -0.-0.j]
 [-0.-0.j -1.-0.j]]

op.num=72
inversion+d
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
[[-1.-0.j -0.-0.j]
 [-0.-0.j -1.-0.j]]



In [158]:
for i in range(len(g8p_full)):
    print("[{}]\noperation:{}\nchar_wavefunction={}\nchar_bilbao={}\n".
         format(i,g6p_full[i].operation,g6p_full[i].char,GM6p[i].char))

[0]
operation:identity
char_wavefunction=(2.000000000000004+0j)
char_bilbao=(2+0j)

[1]
operation:180 deg rotation - cart. axis [0,0,1]
char_wavefunction=4.227229677411515e-12j
char_bilbao=0j

[2]
operation:180 deg rotation - cart. axis [0,1,0]
char_wavefunction=-4.171774037331488e-12j
char_bilbao=0j

[3]
operation:180 deg rotation - cart. axis [1,0,0]
char_wavefunction=-4.4859671533004075e-12j
char_bilbao=0j

[4]
operation:120 deg rotation - cart. axis [1,1,1]
char_wavefunction=(0.9999999999859711-3.200273379633245e-12j)
char_bilbao=(1.0000000000000002+0j)

[5]
operation:120 deg rotation - cart. axis [-1,1,-1]
char_wavefunction=(0.9999999999900968-1.7705836796721997e-12j)
char_bilbao=(1.0000000000000002+0j)

[6]
operation:120 deg rotation - cart. axis [1,-1,-1]
char_wavefunction=(0.9999999999864064+2.582378755278114e-13j)
char_bilbao=(1.0000000000000002+0j)

[7]
operation:120 deg rotation - cart. axis [-1,-1,1]
char_wavefunction=(0.9999999999895575+2.572303481329641e-12j)
char_bilbao=

In [160]:
for i in range(len(matrices)):
    print("\n{}\n{}\n".format(matrices[i].operation,matrices[i].matrix))


identity
[[ 1.00000000e+00+0.00000000e+00j -3.47549212e-16-3.06728716e-16j
  -2.54965599e-16-1.10416198e-15j]
 [-3.47549212e-16+3.06728716e-16j  1.00000000e+00+0.00000000e+00j
   1.77543961e-15-1.98015127e-17j]
 [-2.54965599e-16+1.10416198e-15j  1.77543961e-15+1.98015127e-17j
   1.00000000e+00+0.00000000e+00j]]


180 deg rotation - cart. axis [0,0,1]
[[ 0.3711844 -7.98038444e-17j -0.44712383-4.92669864e-01j
   0.39988951-5.09575514e-01j]
 [-0.44712383+4.92669864e-01j -0.67718177-1.53456959e-16j
   0.0526934 +3.09846632e-01j]
 [ 0.39988951+5.09575514e-01j  0.0526934 -3.09846632e-01j
  -0.69400262-2.55500077e-17j]]


180 deg rotation - cart. axis [0,1,0]
[[-0.96960759-6.76172320e-16j -0.13076323-4.64155955e-02j
  -0.06419978+1.91013315e-01j]
 [-0.13076323+4.64155955e-02j -0.36650538+1.76022115e-16j
  -0.01549814-9.19880528e-01j]
 [-0.06419978-1.91013315e-01j -0.01549814+9.19880528e-01j
   0.33611296-2.37678768e-16j]]


180 deg rotation - cart. axis [1,0,0]
[[-0.40157681+2.40539675e-16j 

In [174]:
for i in range(len(g6p_full)):
    print("[{}]\n{}\n{}".format(i,g6p_full[i].operation,g6p_full[i].matrix))
    unit = (g6p_full[i].matrix).dot(np.conjugate(np.transpose(g6p_full[i].matrix)))
    print("check if unitary: M M^+ = c*I\n{}\n".format(unit))

[0]
identity
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
check if unitary: M M^+ = c*I
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

[1]
180 deg rotation - cart. axis [0,0,1]
[[ 0.        +0.32369328j  0.64300812-0.69409165j]
 [-0.64300812-0.69409165j  0.        -0.32369328j]]
check if unitary: M M^+ = c*I
[[ 1.00000000e+00-1.48442922e-17j -2.93409741e-12-2.71829781e-12j]
 [-2.93409741e-12+2.71829781e-12j  1.00000000e+00-1.58385622e-17j]]

[2]
180 deg rotation - cart. axis [0,1,0]
[[ 0.        +0.71828271j  0.31050028+0.6226231j ]
 [-0.31050028+0.6226231j   0.        -0.71828271j]]
check if unitary: M M^+ = c*I
[[ 1.00000000e+00-4.83071881e-20j -2.59825494e-12+1.29557476e-12j]
 [-2.59825494e-12-1.29557476e-12j  1.00000000e+00+3.06010392e-18j]]

[3]
180 deg rotation - cart. axis [1,0,0]
[[ 0.        -0.61586736j  0.70009294+0.36135476j]
 [-0.70009294+0.36135476j  0.        +0.61586736j]]
check if unitary: M M^+ = c*I
[[ 1.00000000e+00+3.29538683e-18j -1.62120317e-12+3.14093196e-12j]
 [-1.62120317e-12-3.14

In [301]:
def Compute_small_r(sym_table,sym_wf):
    if len(sym_table)!=len(sym_wf):
        raise ValueError("Irreps are not compatible")
    order=len(sym_table)
    print("order={}".format(order))
    dim = len(sym_table[0].matrix)
    print("dim={}".format(dim))
    inv = int(round(order/2))
    r = np.zeros((dim,dim),dtype=complex)
    sq = np.sqrt(dim/order)
    for i in range(dim):
        for j in range(dim):
            s = 0+0j
            for g in range(order):
                s1 = 0+0j
                s1 = sym_table[g].matrix[i,i]*nplin.inv(sym_wf[g].matrix)[j,j]
                s += s1
            r[i,j] = np.sqrt(s)
    return sq*r


r_g6p = Compute_small_r(GM6p,g6p_full)
summ=0
print("r={}".format(r_g6p))
for i in range(len(r_g6p)):
    summ += r_g6p[i,1]**2
print(summ)

order=96
dim=2
r=[[0.58150955+4.48605224e-17j 0.81353958+1.48345796e-17j]
 [0.81353958-3.27766028e-17j 0.58150955-7.82684055e-18j]]
(1.0000000000112066+1.503427012513233e-17j)


In [302]:
print(r)

[[0.82800495+4.19012827e-18j 0.54700237+2.69562809e-17j
  0.1232729 +2.81444426e-17j]
 [0.40175753-8.63567374e-17j 0.72238741+0.00000000e+00j
  0.56280308+2.15760444e-17j]
 [0.39115047-8.86985252e-17j 0.42301871-2.46049183e-16j
  0.81734722-4.08558639e-17j]]


In [303]:
def Gather_Unitary_Transforms(r,sym_table,sym_wf):
    if len(sym_table)!=len(sym_wf):
        raise ValueError("Irreps are not compatible")
    dim = len(r)
    order=len(sym_table)
    print("order={}".format(order))
    print("dim={}".format(dim))
    great_u = []
    for m in range(dim):
        g_u=[]
        for n in range(dim):
            g_u.append(0)
        great_u.append(g_u)
    factor=dim/order
    for a in range(dim):
        for b in range(dim):
            if abs(r[a,b])>1e-14:
                small_u = np.zeros((dim,dim),dtype=complex)
                for i in range(dim):
                    for j in range(dim):
                        u_ij=0+0j
                        for g in range(order):
                            u1 = nplin.inv(sym_wf[g].matrix)[i,a]*sym_table[g].matrix[b,j]
                            u_ij += u1 
                        small_u[i,j]=factor/r[a,b] *u_ij
                great_u[a][b]=np.array(small_u)
            else: 
                great_u[a][b]=0
    #great_u=np.array(great_u)
    return np.array(great_u)
U = Gather_Unitary_Transforms(r_g6p,GM6p,g6p_full)
print(U)

order=96
dim=2
[[[[ 0.58150955+4.48605224e-17j  0.52954192+6.17601813e-01j]
   [ 0.59680159-5.52878384e-01j -0.57768186-6.66109791e-02j]]

  [[ 0.37851101-4.41455296e-01j  0.81353958-8.03877851e-17j]
   [-0.03125507-8.12938965e-01j -0.42658751+3.95191668e-01j]]]


 [[[ 0.42658751+3.95191668e-01j -0.03125507+8.12938965e-01j]
   [ 0.81353958+6.24457620e-17j -0.37851101-4.41455296e-01j]]

  [[-0.57768186+6.66109791e-02j -0.59680159-5.52878384e-01j]
   [-0.52954192+6.17601813e-01j  0.58150955-7.82684055e-18j]]]]


In [304]:
u = U[0,0]
print(u)
print(u.dot(np.conjugate(np.transpose(u))))

[[ 0.58150955+4.48605224e-17j  0.52954192+6.17601813e-01j]
 [ 0.59680159-5.52878384e-01j -0.57768186-6.66109791e-02j]]
[[1.00000000e+00-2.37351615e-17j 1.50662816e-12-4.95103958e-13j]
 [1.50662816e-12+4.95159469e-13j 1.00000000e+00-2.03288074e-17j]]


In [305]:
basis2_up = C_Rso[0,k_start:k_stop+1]
basis2_dn = C_Rso[1,k_start:k_stop+1]
print(basis2_up.shape)

basis2_gm6p_up = u.T.dot(basis2_up)
basis2_gm6p_dn = u.T.dot(basis2_dn)

(2, 16560)


In [308]:
m=0
n=0
A=np.vdot(basis2_gm6p_up[m],basis2_gm6p_up[n])
B=np.vdot(basis2_gm6p_dn[m],basis2_gm6p_dn[n])
print ("{}\n{}\n{}".format(A, B, A+B))
print(u.trace())

(0.9999995489343643+0j)
(4.51080131628274e-07+0j)
(1.000000000014496+0j)
(0.0038276875383557174-0.06661097913394363j)


In [309]:
A=np.vdot(basis4_up[m],basis4_up[n])
B=np.vdot(basis4_dn[m],basis4_dn[n])
print (A, B, A+B)

(0.5529008991750924+0j) (0.44709910082490895+0j) (1.0000000000000013+0j)


In [297]:
summ = 0
i,j,k,l = 0,0,0,0
for m in range(len(g6p_full)):
    psi=(g6p_full[m].matrix)
    v  = np.transpose(np.conjugate(g6p_full[m].matrix))
    summ += psi[i,j]*v[k,l]
    print("[{}]{}".format(m+1,g6p_full[m].operation))
print (summ)

[1]identity
[2]180 deg rotation - cart. axis [0,0,1]
[3]180 deg rotation - cart. axis [0,1,0]
[4]180 deg rotation - cart. axis [1,0,0]
[5]120 deg rotation - cart. axis [1,1,1]
[6]120 deg rotation - cart. axis [-1,1,-1]
[7]120 deg rotation - cart. axis [1,-1,-1]
[8]120 deg rotation - cart. axis [-1,-1,1]
[9]120 deg rotation - cart. axis [-1,-1,-1]
[10]120 deg rotation - cart. axis [-1,1,1]
[11]120 deg rotation - cart. axis [1,1,-1]
[12]120 deg rotation - cart. axis [1,-1,1]
[13]180 deg rotation - cart. axis [1,1,0]
[14]180 deg rotation - cart. axis [1,-1,0]
[15] 90 deg rotation - cart. axis [0,0,-1]
[16] 90 deg rotation - cart. axis [0,0,1]
[17] 90 deg rotation - cart. axis [-1,0,0]
[18]180 deg rotation - cart. axis [0,1,1]
[19]180 deg rotation - cart. axis [0,1,-1]
[20] 90 deg rotation - cart. axis [1,0,0]
[21] 90 deg rotation - cart. axis [0,1,0]
[22]180 deg rotation - cart. axis [1,0,1]
[23] 90 deg rotation - cart. axis [0,-1,0]
[24]180 deg rotation - cart. axis [-1,0,1]
[25]inversio

In [295]:
sym_path="/home/mjocic/Post_processing/Symmetry_groups"
file_name="Oh_double2_bilbao_order.xml"
symmetries_double,sig_double=Gather_Symmetries_double(sym_path,file_name)

k_start=50
k_stop=51
g6p_full=Gather_Matrices_double(
    k_start,k_stop,symmetries_double,sig_double,
    G_Rso,C_Rso[0],C_Rso[1],k_Rpoint,details=True,all_elements=True)

Dimension of 3x3x3 square matrix is: 33
Creating G-matrix
G-matrix: Done

Now performing for identity

Symm.Op. matrix:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Spin.Op. matrix:
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Matrix = 
 [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

Now performing for 180 deg rotation - cart. axis [0,0,1]

Symm.Op. matrix:
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.-1.j  0.+0.j]
 [ 0.+0.j  0.+1.j]]
Matrix = 
 [[ 0.        +0.32369328j  0.64300812-0.69409165j]
 [-0.64300812-0.69409165j  0.        -0.32369328j]]

Now performing for 180 deg rotation - cart. axis [0,1,0]

Symm.Op. matrix:
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
Spin.Op. matrix:
[[ 0.+0.j -1.+0.j]
 [ 1.+0.j  0.+0.j]]
Matrix = 
 [[ 0.        +0.71828271j  0.31050028+0.6226231j ]
 [-0.31050028+0.6226231j   0.        -0.71828271j]]

Now performing for 180 deg rotation - cart. axis [1,0,0]

Symm.Op. matrix:
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
Spin.Op. matrix:
[[ 0.+0.j -0.-1.j]
 [-0.-1.j  0.+0.j]]
Matrix = 
 [[ 0.   

Matrix = 
 [[ 0.        -0.61586736j  0.70009294+0.36135476j]
 [-0.70009294+0.36135476j  0.        +0.61586736j]]

Now performing for inv. 120 deg rotation - cart. axis [1,1,1]

Symm.Op. matrix:
[[ 0  0 -1]
 [-1  0  0]
 [ 0 -1  0]]
Spin.Op. matrix:
[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]
Matrix = 
 [[ 0.5       +0.21305431j  0.82680067+0.14494311j]
 [-0.82680067+0.14494311j  0.5       -0.21305431j]]

Now performing for inv. 120 deg rotation - cart. axis [-1,1,-1]

Symm.Op. matrix:
[[ 0  0 -1]
 [ 1  0  0]
 [ 0  1  0]]
Spin.Op. matrix:
[[ 0.5+0.5j -0.5+0.5j]
 [ 0.5+0.5j  0.5-0.5j]]
Matrix = 
 [[ 0.5       +0.5052284j  -0.51630039+0.47767999j]
 [ 0.51630039+0.47767999j  0.5       -0.5052284j ]]

Now performing for inv. 120 deg rotation - cart. axis [1,-1,-1]

Symm.Op. matrix:
[[ 0  0  1]
 [ 1  0  0]
 [ 0 -1  0]]
Spin.Op. matrix:
[[ 0.5+0.5j  0.5-0.5j]
 [-0.5-0.5j  0.5-0.5j]]
Matrix = 
 [[ 0.5       -0.82892168j -0.12670773+0.21641165j]
 [ 0.12670773+0.21641165j  0.5       +0.828921

Matrix = 
 [[-0.5       +0.82892168j  0.12670773-0.21641165j]
 [-0.12670773-0.21641165j -0.5       -0.82892168j]]

Now performing for 120 deg rotation - cart. axis [-1,-1,1]+d

Symm.Op. matrix:
[[ 0  0 -1]
 [ 1  0  0]
 [ 0 -1  0]]
Spin.Op. matrix:
[[-0.5+0.5j -0.5-0.5j]
 [ 0.5-0.5j -0.5-0.5j]]
Matrix = 
 [[-0.5       -0.11063897j  0.18379255+0.83903475j]
 [-0.18379255+0.83903475j -0.5       +0.11063897j]]

Now performing for 120 deg rotation - cart. axis [-1,-1,-1]+d

Symm.Op. matrix:
[[0 1 0]
 [0 0 1]
 [1 0 0]]
Spin.Op. matrix:
[[-0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j -0.5+0.5j]]
Matrix = 
 [[-0.5       +0.21305431j  0.82680067+0.14494311j]
 [-0.82680067+0.14494311j -0.5       -0.21305431j]]

Now performing for 120 deg rotation - cart. axis [-1,1,1]+d

Symm.Op. matrix:
[[ 0 -1  0]
 [ 0  0  1]
 [-1  0  0]]
Spin.Op. matrix:
[[-0.5+0.5j  0.5-0.5j]
 [-0.5-0.5j -0.5-0.5j]]
Matrix = 
 [[-0.5       -0.82892168j -0.12670773+0.21641165j]
 [ 0.12670773+0.21641165j -0.5       +0.82892168j]]

Now perfo

Matrix = 
 [[-0.5       -0.82892168j -0.12670773+0.21641165j]
 [ 0.12670773+0.21641165j -0.5       +0.82892168j]]

Now performing for inv. 120 deg rotation - cart. axis [1,1,-1]+d

Symm.Op. matrix:
[[ 0 -1  0]
 [ 0  0  1]
 [ 1  0  0]]
Spin.Op. matrix:
[[-0.5-0.5j  0.5+0.5j]
 [-0.5+0.5j -0.5+0.5j]]
Matrix = 
 [[-0.5       +0.11063897j -0.18379255-0.83903475j]
 [ 0.18379255-0.83903475j -0.5       -0.11063897j]]

Now performing for inv. 120 deg rotation - cart. axis [1,-1,1]+d

Symm.Op. matrix:
[[ 0  1  0]
 [ 0  0  1]
 [-1  0  0]]
Spin.Op. matrix:
[[-0.5+0.5j -0.5+0.5j]
 [ 0.5+0.5j -0.5-0.5j]]
Matrix = 
 [[-0.5       +0.5052284j  -0.51630039+0.47767999j]
 [ 0.51630039+0.47767999j -0.5       -0.5052284j ]]

Now performing for inv. 180 deg rotation - cart. axis [1,1,0]+d

Symm.Op. matrix:
[[ 0 -1  0]
 [-1  0  0]
 [ 0  0  1]]
Spin.Op. matrix:
[[-0.        -0.j          0.70710678+0.70710678j]
 [-0.70710678+0.70710678j -0.        -0.j        ]]
Matrix = 
 [[ 0.        -0.07241858j -0.71459732